# Inclusion of the Malawi survey

Columns we need:

weight --> here, the sum over all weights seems to represent the number of hh (not people)

income_col --> needs to be worked out here from the sectors

income_sp --> needs to be elaborated

region --> is given in "hh_a01"

decile --> need to be calculated

savings --> need to be calculated

poverty_line --> needs to be defined 

ispoor --> ?

isurban --> in "reside"

vul --> can be extracted from module hh F


Note: The survey data from the panel run seems to be sampled with a lower sampling rate, the data does not seem to be representative on the district level any more, so we work with the full-sample data.

In [1]:
import pandas as pd
import numpy as np

In [2]:
general_info = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Household/hh_mod_a_filt.csv')
general_info = general_info.iloc[:,:6]
general_info = general_info.drop(columns=['qx_type'])
general_info

,case_id,ea_id,reside,hh_wgt,hh_a01
0,101010160009,10101016,2,126.560097,101
1,101010160033,10101016,2,126.560097,101
2,101010160060,10101016,2,126.560097,101
3,101010160068,10101016,2,126.560097,101
4,101010160069,10101016,2,126.560097,101
...,...,...,...,...,...
12266,315556140342,31555614,1,456.450592,315
12267,315556140365,31555614,1,456.450592,315
12268,315556140388,31555614,1,456.450592,315
12269,315556140411,31555614,1,456.450592,315


In [3]:
general_info.columns = ['case_id', 'ea_id', 'isurban', 'weight', 'region']

In [4]:
general_info

,case_id,ea_id,isurban,weight,region
0,101010160009,10101016,2,126.560097,101
1,101010160033,10101016,2,126.560097,101
2,101010160060,10101016,2,126.560097,101
3,101010160068,10101016,2,126.560097,101
4,101010160069,10101016,2,126.560097,101
...,...,...,...,...,...
12266,315556140342,31555614,1,456.450592,315
12267,315556140365,31555614,1,456.450592,315
12268,315556140388,31555614,1,456.450592,315
12269,315556140411,31555614,1,456.450592,315


In [5]:
general_info['weight'].sum()

3072524.951984406

In [6]:
general_info.loc[general_info['region']==101]

,case_id,ea_id,isurban,weight,region
0,101010160009,10101016,2,126.560097,101
1,101010160033,10101016,2,126.560097,101
2,101010160060,10101016,2,126.560097,101
3,101010160068,10101016,2,126.560097,101
4,101010160069,10101016,2,126.560097,101
...,...,...,...,...,...
379,101202880104,10120288,1,59.150002,101
380,101202880118,10120288,1,59.150002,101
381,101202880125,10120288,1,59.150002,101
382,101202880132,10120288,1,59.150002,101


In [7]:
case_ids = list(general_info['case_id'])
case_ids

[101010160009,
 101010160033,
 101010160060,
 101010160068,
 101010160069,
 101010160070,
 101010160074,
 101010160086,
 101010160096,
 101010160143,
 101010160159,
 101010160180,
 101010160207,
 101010160235,
 101010160254,
 101010160265,
 101012040003,
 101012040007,
 101012040011,
 101012040020,
 101012040024,
 101012040028,
 101012040037,
 101012040041,
 101012040045,
 101012040054,
 101012040058,
 101012040062,
 101012040071,
 101012040075,
 101012040079,
 101012040087,
 101012150001,
 101012150004,
 101012150021,
 101012150022,
 101012150026,
 101012150028,
 101012150034,
 101012150038,
 101012150040,
 101012150045,
 101012150048,
 101012150049,
 101012150053,
 101012150059,
 101012150060,
 101012150074,
 101012190012,
 101012190018,
 101012190025,
 101012190032,
 101012190045,
 101012190052,
 101012190059,
 101012190065,
 101012190079,
 101012190086,
 101012190092,
 101012190099,
 101012190112,
 101012190119,
 101012190126,
 101012190139,
 101013550014,
 101013550023,
 101013550

Here, we use district, reside and hh_wgt, but need to check whether this weight refers to the country, region or district.

--> The weights seem to represent only the full number of hh, not the representation of the entire population

For income data we need to move to the individual level:

# Income from labour

Income from labour is given in module E. There is income from a main job and a secondary job and income from ganyu labour. The payent received covers different time periods (week, day, month) and the time worked in each job is also given. So we estimate the salary per year for each job.

In [8]:
ind_data = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Household/hh_mod_e.csv')
ind_data = ind_data[['case_id', 'hh_e22', 'hh_e23', 'hh_e24', 'hh_e25', 'hh_e26a', 'hh_e26b','hh_e27','hh_e28a', 'hh_e28b',
          'hh_e36', 'hh_e37', 'hh_e38', 'hh_e39', 'hh_e40a', 'hh_e40b', 'hh_e41', 'hh_e42a', 'hh_e42b',
          'hh_e55', 'hh_e56', 'hh_e57', 'hh_e58', 'hh_e59']]

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (65) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
ind_data['hh_e26_factor'] = np.nan
ind_data['hh_e28_factor'] = np.nan
ind_data['hh_e40_factor'] = np.nan
ind_data['hh_e42_factor'] = np.nan

In [10]:
# Main job: We insert a factor the payment is multiplied with depending on the time covered by the salary
#payment
ind_data.loc[ind_data['hh_e26b']==3,'hh_e26_factor']= 7
ind_data.loc[ind_data['hh_e26b']==4,'hh_e26_factor']= 1
ind_data.loc[ind_data['hh_e26b']==5,'hh_e26_factor']= 0.25

#allowances
ind_data.loc[ind_data['hh_e28b']==3,'hh_e28_factor']= 7
ind_data.loc[ind_data['hh_e28b']==4,'hh_e28_factor']= 1
ind_data.loc[ind_data['hh_e26b']==5,'hh_e28_factor']= 0.25

#payment
ind_data.loc[ind_data['hh_e40b']==3,'hh_e40_factor']= 7
ind_data.loc[ind_data['hh_e40b']==4,'hh_e40_factor']= 1
ind_data.loc[ind_data['hh_e40b']==5,'hh_e40_factor']= 0.25

#allowances
ind_data.loc[ind_data['hh_e42b']==3,'hh_e42_factor']= 7
ind_data.loc[ind_data['hh_e42b']==4,'hh_e42_factor']= 1
ind_data.loc[ind_data['hh_e42b']==5,'hh_e42_factor']= 0.25

In [11]:
# Provide weekly and annual income
ind_data['inc_lab_wkly_main'] = ind_data['hh_e25']*(ind_data['hh_e26_factor']/ind_data['hh_e26a'])
ind_data['inc_lab_wkly_add_main'] = ind_data['hh_e27']*(ind_data['hh_e28_factor']/ind_data['hh_e28a'])
ind_data['inc_lab_wkly_sec'] = ind_data['hh_e39']*(ind_data['hh_e40_factor']/ind_data['hh_e40a'])
ind_data['inc_lab_wkly_add_sec'] = ind_data['hh_e41']*(ind_data['hh_e42_factor']/ind_data['hh_e42a'])

ind_data['inc_lab_ann_main'] = ind_data['inc_lab_wkly_main'] * ind_data['hh_e23'] * ind_data['hh_e22']    
ind_data['inc_lab_ann_add_main'] = ind_data['inc_lab_wkly_add_main'] * ind_data['hh_e23'] * ind_data['hh_e22']   
ind_data['inc_lab_ann_sec'] = ind_data['inc_lab_wkly_sec'] * ind_data['hh_e36'] * ind_data['hh_e37']  
ind_data['inc_lab_ann_add_sec'] = ind_data['inc_lab_wkly_add_sec'] * ind_data['hh_e36'] * ind_data['hh_e37']

# include ganyu labour
ind_data['inc_lab_wkly_ganyu'] = ind_data['hh_e59'] * ind_data['hh_e58'] 
ind_data['inc_lab_ann_ganyu'] = ind_data['inc_lab_wkly_ganyu'] * ind_data['hh_e57'] * ind_data['hh_e56']

In [12]:
ind_data['inc_lab_wkly_ganyu'].dropna()

cols = ['inc_lab_wkly_main', 'inc_lab_wkly_add_main', 'inc_lab_wkly_sec', 'inc_lab_wkly_add_sec', 'inc_lab_ann_main',
        'inc_lab_ann_add_main', 'inc_lab_ann_sec', 'inc_lab_ann_add_sec', 'inc_lab_wkly_ganyu', 'inc_lab_ann_ganyu']


In [13]:
income_data = ind_data.groupby(['case_id'])\
                                    [cols].apply(sum)\
                                    .reset_index()

In [14]:
income_data['inc_lab_ann']= income_data['inc_lab_ann_main']+income_data['inc_lab_ann_add_main']+income_data['inc_lab_ann_sec']+\
                            income_data['inc_lab_ann_add_sec']+income_data['inc_lab_ann_ganyu']


In [15]:
income_data

,case_id,inc_lab_wkly_main,inc_lab_wkly_add_main,inc_lab_wkly_sec,inc_lab_wkly_add_sec,inc_lab_ann_main,inc_lab_ann_add_main,inc_lab_ann_sec,inc_lab_ann_add_sec,inc_lab_wkly_ganyu,inc_lab_ann_ganyu,inc_lab_ann
0,101010160009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600.0,2400.0,2400.0
1,101010160033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,101010160060,1375.0,0.0,0.0,0.0,66000.0,0.0,0.0,0.0,2060.0,18480.0,84480.0
3,101010160068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1510.0,16080.0,16080.0
4,101010160069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1050.0,2100.0,2100.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12266,315556140342,1875.0,0.0,0.0,0.0,90000.0,0.0,0.0,0.0,0.0,0.0,90000.0
12267,315556140365,1750.0,0.0,0.0,0.0,84000.0,0.0,0.0,0.0,420.0,1680.0,85680.0
12268,315556140388,1500.0,0.0,0.0,0.0,72000.0,0.0,0.0,0.0,0.0,0.0,72000.0
12269,315556140411,3500.0,0.0,0.0,0.0,91000.0,0.0,0.0,0.0,0.0,0.0,91000.0


In [16]:
general_info['income_labour'] = income_data['inc_lab_ann']
general_info

,case_id,ea_id,isurban,weight,region,income_labour
0,101010160009,10101016,2,126.560097,101,2400.0
1,101010160033,10101016,2,126.560097,101,0.0
2,101010160060,10101016,2,126.560097,101,84480.0
3,101010160068,10101016,2,126.560097,101,16080.0
4,101010160069,10101016,2,126.560097,101,2100.0
...,...,...,...,...,...,...
12266,315556140342,31555614,1,456.450592,315,90000.0
12267,315556140365,31555614,1,456.450592,315,85680.0
12268,315556140388,31555614,1,456.450592,315,72000.0
12269,315556140411,31555614,1,456.450592,315,91000.0


# Income from other sources

## Income from enterprises

To extract the incomes from enterprises we try to use the monthly profit weighted by the share of profit that is kept by the household --> this approach does not work as the shares are mostly invalid!

We take the profit and devide it by the number of co-owners!

In [95]:
ep_data = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Household/hh_mod_n2.csv')

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Given:
- the amount of the sales(low, high, average) from the last months (March 2009-March 2011)
- the sales of the last month
- the profit of the last month
- the sales corresponding to high, low and average amounts

Approach:

We estimate the monthly profit for high, low and anverage sales and calculate an annual profit from the sales of each month in operation.

In [96]:
ep_data = ep_data[['case_id', 'hh_n13a', 'hh_n13b', 'hh_n25a', 'hh_n25b', 'hh_n25c', 'hh_n25d', 'hh_n25e', 'hh_n25f', 'hh_n25g',
                   'hh_n25h', 'hh_n25i', 'hh_n25j', 'hh_n25k', 'hh_n25l', 'hh_n25m', 'hh_n25n', 'hh_n25o', 'hh_n25p',
                   'hh_n25q', 'hh_n25r', 'hh_n25s', 'hh_n25t', 'hh_n25u', 'hh_n25v', 'hh_n25w','hh_n25x','hh_n25y','hh_n32', 'hh_n33', 'hh_n34', 'hh_n35',
                   'hh_n36', 'hh_n37', 'hh_n38', 'hh_n39', 'hh_n40']]


In [97]:
ep_data.loc[:,'low_sales_fac'] = np.nan
ep_data.loc[:,'high_sales_fac'] = np.nan

We take the estimate factors for a high low in dependence of the average sale (1 = average sale).
The category of sales (high, low and average) of the last month is given in 'hh_n33' the amount gained corresponding to the category is given in 'hh_n32'. The amounts for the two other categories are then given in 'hh_n34 - hh_n39' depending on which sales category belongs to the last month. (if hh_n33 = 1 --> hh_n34 and hh_n35, if hh_n33 = 2 --> hh_n36 and hh_n37, if hh_n33 = 3 --> hh_n38 and hh_n39)

In [98]:
ep_data.loc[ep_data['hh_n33']==1, 'low_sales_fac'] = ep_data.loc[ep_data['hh_n33']==1, 'hh_n32']/ep_data.loc[ep_data['hh_n33']==1, 'hh_n34']
ep_data.loc[ep_data['hh_n33']==2, 'low_sales_fac'] = ep_data.loc[ep_data['hh_n33']==2, 'hh_n36']/ep_data.loc[ep_data['hh_n33']==2, 'hh_n32']
ep_data.loc[ep_data['hh_n33']==3, 'low_sales_fac'] = ep_data.loc[ep_data['hh_n33']==3, 'hh_n38']/ep_data.loc[ep_data['hh_n33']==3, 'hh_n39']

ep_data.loc[ep_data['hh_n33']==1, 'high_sales_fac'] = ep_data.loc[ep_data['hh_n33']==1, 'hh_n35']/ep_data.loc[ep_data['hh_n33']==1, 'hh_n34']
ep_data.loc[ep_data['hh_n33']==2, 'high_sales_fac'] = ep_data.loc[ep_data['hh_n33']==2, 'hh_n37']/ep_data.loc[ep_data['hh_n33']==2, 'hh_n32']
ep_data.loc[ep_data['hh_n33']==3, 'high_sales_fac'] = ep_data.loc[ep_data['hh_n33']==3, 'hh_n32']/ep_data.loc[ep_data['hh_n33']==3, 'hh_n39']

In [99]:
# Check for infinite values
ep_data.loc[(ep_data['low_sales_fac']==np.inf)].iloc[:,25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
5,NaN,NaN,NaN,16000.0,3.0,NaN,NaN,NaN,NaN,12000.0,0.0,7200.0,inf,inf
40,NaN,NaN,NaN,3000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,140.0,inf,NaN
43,NaN,NaN,NaN,22300.0,3.0,NaN,NaN,NaN,NaN,11800.0,0.0,3500.0,inf,inf
163,NaN,NaN,NaN,6300.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,2300.0,inf,NaN
244,0.0,0.0,NaN,10000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,3000.0,inf,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9494,NaN,NaN,NaN,1000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,300.0,inf,NaN
10043,NaN,NaN,NaN,5000.0,3.0,NaN,NaN,NaN,NaN,2000.0,0.0,3000.0,inf,inf
10044,NaN,NaN,NaN,3000.0,3.0,NaN,NaN,NaN,NaN,1000.0,0.0,2000.0,inf,inf
11455,NaN,NaN,NaN,2500.0,3.0,NaN,NaN,NaN,NaN,2500.0,0.0,500.0,inf,inf


There are some rows which infinite values in 'low_sales_fac' and see that there are sometimes zeros in the columns of other sale categories. We need to replace these zeros by estimating the amounts from the amount from the last month.

In [100]:
# We start with the case that the last month was one of low sales
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==1.0)].iloc[:,25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
40,NaN,NaN,NaN,3000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,140.0,inf,NaN
163,NaN,NaN,NaN,6300.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,2300.0,inf,NaN
244,0.0,0.0,NaN,10000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,3000.0,inf,NaN
427,NaN,NaN,NaN,7000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,4400.0,inf,NaN
442,NaN,NaN,NaN,5200.0,1.0,0.0,10400.0,NaN,NaN,NaN,NaN,32400.0,inf,inf
456,NaN,NaN,NaN,7000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,4000.0,inf,NaN
510,NaN,NaN,NaN,1000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,300.0,inf,NaN
514,NaN,NaN,NaN,1000.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,500.0,inf,NaN
545,NaN,NaN,NaN,800.0,1.0,0.0,2000.0,NaN,NaN,NaN,NaN,300.0,inf,inf
902,NaN,NaN,NaN,17000.0,1.0,0.0,17000.0,NaN,NaN,NaN,NaN,9000.0,inf,inf


We see that sometimes only hh_n34 and sometimes both 'hh_n34' and 'hh_n35' are missing.
- 1st case both are missing we put the value in hh_n32 for both
- 2nd case only hh_n34 is missing we estimate it as the average between 'hh_n32' and 'hh_n35'

In [101]:
# First case
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==1.0)&(ep_data['hh_n35']==0.),['hh_n34','hh_n35']]=ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==1.0)&(ep_data['hh_n35']==0.),'hh_n32']

In [102]:
# Check
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==1.0)].iloc[:,25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
40,NaN,NaN,NaN,3000.0,1.0,3000.0,3000.0,NaN,NaN,NaN,NaN,140.0,inf,NaN
163,NaN,NaN,NaN,6300.0,1.0,6300.0,6300.0,NaN,NaN,NaN,NaN,2300.0,inf,NaN
244,0.0,0.0,NaN,10000.0,1.0,10000.0,10000.0,NaN,NaN,NaN,NaN,3000.0,inf,NaN
427,NaN,NaN,NaN,7000.0,1.0,7000.0,7000.0,NaN,NaN,NaN,NaN,4400.0,inf,NaN
442,NaN,NaN,NaN,5200.0,1.0,0.0,10400.0,NaN,NaN,NaN,NaN,32400.0,inf,inf
456,NaN,NaN,NaN,7000.0,1.0,7000.0,7000.0,NaN,NaN,NaN,NaN,4000.0,inf,NaN
510,NaN,NaN,NaN,1000.0,1.0,1000.0,1000.0,NaN,NaN,NaN,NaN,300.0,inf,NaN
514,NaN,NaN,NaN,1000.0,1.0,1000.0,1000.0,NaN,NaN,NaN,NaN,500.0,inf,NaN
545,NaN,NaN,NaN,800.0,1.0,0.0,2000.0,NaN,NaN,NaN,NaN,300.0,inf,inf
902,NaN,NaN,NaN,17000.0,1.0,0.0,17000.0,NaN,NaN,NaN,NaN,9000.0,inf,inf


In [103]:
# Second case
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==1.0)&(ep_data['hh_n34']==0.), 'hh_n34']= (ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==1.0)&(ep_data['hh_n34']==0.),'hh_n35']+\
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==1.0)&(ep_data['hh_n34']==0.),'hh_n32'])/2

In [104]:
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==1.0)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
40,NaN,NaN,NaN,3000.0,1.0,3000.0,3000.0,NaN,NaN,NaN,NaN,140.0,inf,NaN
163,NaN,NaN,NaN,6300.0,1.0,6300.0,6300.0,NaN,NaN,NaN,NaN,2300.0,inf,NaN
244,0.0,0.0,NaN,10000.0,1.0,10000.0,10000.0,NaN,NaN,NaN,NaN,3000.0,inf,NaN
427,NaN,NaN,NaN,7000.0,1.0,7000.0,7000.0,NaN,NaN,NaN,NaN,4400.0,inf,NaN
442,NaN,NaN,NaN,5200.0,1.0,7800.0,10400.0,NaN,NaN,NaN,NaN,32400.0,inf,inf
456,NaN,NaN,NaN,7000.0,1.0,7000.0,7000.0,NaN,NaN,NaN,NaN,4000.0,inf,NaN
510,NaN,NaN,NaN,1000.0,1.0,1000.0,1000.0,NaN,NaN,NaN,NaN,300.0,inf,NaN
514,NaN,NaN,NaN,1000.0,1.0,1000.0,1000.0,NaN,NaN,NaN,NaN,500.0,inf,NaN
545,NaN,NaN,NaN,800.0,1.0,1400.0,2000.0,NaN,NaN,NaN,NaN,300.0,inf,inf
902,NaN,NaN,NaN,17000.0,1.0,17000.0,17000.0,NaN,NaN,NaN,NaN,9000.0,inf,inf


In [105]:
# We continue with the case that the last month was one of average sales
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==2.0)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
11948,NaN,NaN,NaN,0.0,2.0,NaN,NaN,500.0,900.0,NaN,NaN,400.0,inf,inf


In [106]:
# we only need to estimate the average sales from the high and low ones
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==2.0),'hh_n32']=(ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==2.0),'hh_n36']+\
                                                                                  ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==2.0),'hh_n37'])/2

In [107]:
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==2.0)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
11948,NaN,NaN,NaN,700.0,2.0,NaN,NaN,500.0,900.0,NaN,NaN,400.0,inf,inf


In [108]:
# ... and with the case that the last month was one of high sales
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
5,NaN,NaN,NaN,16000.0,3.0,NaN,NaN,NaN,NaN,12000.0,0.0,7200.0,inf,inf
43,NaN,NaN,NaN,22300.0,3.0,NaN,NaN,NaN,NaN,11800.0,0.0,3500.0,inf,inf
291,NaN,NaN,NaN,18000.0,3.0,NaN,NaN,NaN,NaN,7200.0,0.0,12000.0,inf,inf
295,NaN,NaN,NaN,16500.0,3.0,NaN,NaN,NaN,NaN,9000.0,0.0,1500.0,inf,inf
349,NaN,NaN,NaN,15000.0,3.0,NaN,NaN,NaN,NaN,3000.0,0.0,11000.0,inf,inf
358,NaN,NaN,NaN,32000.0,3.0,NaN,NaN,NaN,NaN,18000.0,0.0,10000.0,inf,inf
361,NaN,NaN,NaN,25000.0,3.0,NaN,NaN,NaN,NaN,15000.0,0.0,5000.0,inf,inf
546,NaN,NaN,NaN,12000.0,3.0,NaN,NaN,NaN,NaN,4000.0,0.0,5000.0,inf,inf
695,NaN,NaN,NaN,750.0,3.0,NaN,NaN,NaN,NaN,500.0,0.0,1750.0,inf,inf
753,NaN,NaN,NaN,18000.0,3.0,NaN,NaN,NaN,NaN,6000.0,0.0,11000.0,inf,inf


In [109]:
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0),'hh_n39']=(ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0),'hh_n38']+\
                                                                                   ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0),'hh_n32'])/2

In [110]:
ep_data.loc[(ep_data['low_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
5,NaN,NaN,NaN,16000.0,3.0,NaN,NaN,NaN,NaN,12000.0,14000.0,7200.0,inf,inf
43,NaN,NaN,NaN,22300.0,3.0,NaN,NaN,NaN,NaN,11800.0,17050.0,3500.0,inf,inf
291,NaN,NaN,NaN,18000.0,3.0,NaN,NaN,NaN,NaN,7200.0,12600.0,12000.0,inf,inf
295,NaN,NaN,NaN,16500.0,3.0,NaN,NaN,NaN,NaN,9000.0,12750.0,1500.0,inf,inf
349,NaN,NaN,NaN,15000.0,3.0,NaN,NaN,NaN,NaN,3000.0,9000.0,11000.0,inf,inf
358,NaN,NaN,NaN,32000.0,3.0,NaN,NaN,NaN,NaN,18000.0,25000.0,10000.0,inf,inf
361,NaN,NaN,NaN,25000.0,3.0,NaN,NaN,NaN,NaN,15000.0,20000.0,5000.0,inf,inf
546,NaN,NaN,NaN,12000.0,3.0,NaN,NaN,NaN,NaN,4000.0,8000.0,5000.0,inf,inf
695,NaN,NaN,NaN,750.0,3.0,NaN,NaN,NaN,NaN,500.0,625.0,1750.0,inf,inf
753,NaN,NaN,NaN,18000.0,3.0,NaN,NaN,NaN,NaN,6000.0,12000.0,11000.0,inf,inf


In [111]:
# check the high_sales_fac
ep_data.loc[(ep_data['high_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0)
           &(ep_data['hh_n38']==0.0)&(ep_data['hh_n39']==0.0)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
44,NaN,NaN,NaN,18000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,9400.0,NaN,inf
665,NaN,NaN,NaN,17000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,7000.0,NaN,inf
1609,NaN,NaN,NaN,10000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,4200.0,NaN,inf
1794,NaN,NaN,NaN,4000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,3000.0,NaN,inf
1878,NaN,NaN,NaN,52000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,20000.0,NaN,inf
2049,NaN,NaN,NaN,40000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,inf
2120,NaN,NaN,NaN,5000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,3000.0,NaN,inf
2182,NaN,NaN,NaN,73000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,28000.0,NaN,inf
2202,NaN,NaN,NaN,11000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,6000.0,NaN,inf
2264,NaN,NaN,NaN,1400000.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,400000.0,NaN,inf


In [112]:
ep_data.loc[(ep_data['high_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0)
            &(ep_data['hh_n38']==0.0)&(ep_data['hh_n39']==0.0), ['hh_n38', 'hh_n39']]=ep_data.loc[(ep_data['high_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0), 'hh_n32']

In [113]:
ep_data.loc[(ep_data['high_sales_fac']==np.inf)&(ep_data['hh_n33']==3.0)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac
5,NaN,NaN,NaN,16000.0,3.0,NaN,NaN,NaN,NaN,12000.0,14000.0,7200.0,inf,inf
43,NaN,NaN,NaN,22300.0,3.0,NaN,NaN,NaN,NaN,11800.0,17050.0,3500.0,inf,inf
44,NaN,NaN,NaN,18000.0,3.0,NaN,NaN,NaN,NaN,18000.0,18000.0,9400.0,NaN,inf
291,NaN,NaN,NaN,18000.0,3.0,NaN,NaN,NaN,NaN,7200.0,12600.0,12000.0,inf,inf
295,NaN,NaN,NaN,16500.0,3.0,NaN,NaN,NaN,NaN,9000.0,12750.0,1500.0,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11844,0.0,0.0,3.0,5600.0,3.0,NaN,NaN,NaN,NaN,5600.0,5600.0,3000.0,NaN,inf
11848,0.0,3.0,NaN,1500.0,3.0,NaN,NaN,NaN,NaN,1500.0,1500.0,500.0,NaN,inf
11976,NaN,NaN,NaN,10000.0,3.0,NaN,NaN,NaN,NaN,10000.0,10000.0,8000.0,NaN,inf
12021,NaN,NaN,NaN,800.0,3.0,NaN,NaN,NaN,NaN,800.0,800.0,300.0,NaN,inf


In [114]:
ep_data.loc[ep_data['hh_n33']==1, 'low_sales_fac'] = ep_data.loc[ep_data['hh_n33']==1, 'hh_n32']/ep_data.loc[ep_data['hh_n33']==1, 'hh_n34']
ep_data.loc[ep_data['hh_n33']==2, 'low_sales_fac'] = ep_data.loc[ep_data['hh_n33']==2, 'hh_n36']/ep_data.loc[ep_data['hh_n33']==2, 'hh_n32']
ep_data.loc[ep_data['hh_n33']==3, 'low_sales_fac'] = ep_data.loc[ep_data['hh_n33']==3, 'hh_n38']/ep_data.loc[ep_data['hh_n33']==3, 'hh_n39']

ep_data.loc[ep_data['hh_n33']==1, 'high_sales_fac'] = ep_data.loc[ep_data['hh_n33']==1, 'hh_n35']/ep_data.loc[ep_data['hh_n33']==1, 'hh_n34']
ep_data.loc[ep_data['hh_n33']==2, 'high_sales_fac'] = ep_data.loc[ep_data['hh_n33']==2, 'hh_n37']/ep_data.loc[ep_data['hh_n33']==2, 'hh_n32']
ep_data.loc[ep_data['hh_n33']==3, 'high_sales_fac'] = ep_data.loc[ep_data['hh_n33']==3, 'hh_n32']/ep_data.loc[ep_data['hh_n33']==3, 'hh_n39']


In [115]:
# check
ep_data.loc[(ep_data['low_sales_fac']==np.inf)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac


In [116]:
ep_data.loc[(ep_data['high_sales_fac']==np.inf)].iloc[:, 25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac


In [117]:
ep_data.loc[:,'low_profit'] = np.nan
ep_data.loc[:,'high_profit'] = np.nan
ep_data.loc[:,'average_profit'] = np.nan

In [118]:
ep_data.loc[ep_data['hh_n33']==1, 'low_profit'] = ep_data.loc[ep_data['hh_n33']==1, 'hh_n40']
ep_data.loc[ep_data['hh_n33']==1, 'average_profit'] = ep_data.loc[ep_data['hh_n33']==1, 'hh_n40']/ep_data.loc[ep_data['hh_n33']==1, 'low_sales_fac']
ep_data.loc[ep_data['hh_n33']==1, 'high_profit'] = ep_data.loc[ep_data['hh_n33']==1, 'average_profit']*ep_data.loc[ep_data['hh_n33']==1, 'high_sales_fac']

ep_data.loc[ep_data['hh_n33']==2, 'average_profit'] = ep_data.loc[ep_data['hh_n33']==2, 'hh_n40']
ep_data.loc[ep_data['hh_n33']==2, 'low_profit'] = ep_data.loc[ep_data['hh_n33']==2, 'hh_n40']*ep_data.loc[ep_data['hh_n33']==2, 'low_sales_fac']
ep_data.loc[ep_data['hh_n33']==2, 'high_profit'] = ep_data.loc[ep_data['hh_n33']==2, 'average_profit']*ep_data.loc[ep_data['hh_n33']==2, 'high_sales_fac']

ep_data.loc[ep_data['hh_n33']==3, 'high_profit'] = ep_data.loc[ep_data['hh_n33']==3, 'hh_n40']
ep_data.loc[ep_data['hh_n33']==3, 'average_profit'] = ep_data.loc[ep_data['hh_n33']==3, 'hh_n40']/ep_data.loc[ep_data['hh_n33']==3, 'high_sales_fac']
ep_data.loc[ep_data['hh_n33']==3, 'low_profit'] = ep_data.loc[ep_data['hh_n33']==3, 'average_profit']*ep_data.loc[ep_data['hh_n33']==3, 'low_sales_fac']

Finally, we check whether there are negative profits

In [119]:
ep_data.loc[ep_data['hh_n40']<0].iloc[:,25:]

,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,hh_n34,hh_n35,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac,low_profit,high_profit,average_profit
2,NaN,NaN,NaN,700.0,1.0,1500.0,2500.0,NaN,NaN,NaN,NaN,-350.0,0.466667,1.666667,-350.0,-1250.000000,-750.000000
3310,NaN,NaN,NaN,1500.0,1.0,2000.0,4000.0,NaN,NaN,NaN,NaN,-500.0,0.750000,2.000000,-500.0,-1333.333333,-666.666667
3907,NaN,NaN,NaN,5000.0,1.0,6000.0,8500.0,NaN,NaN,NaN,NaN,-7200.0,0.833333,1.416667,-7200.0,-12240.000000,-8640.000000


All negative profits belong to negative profits.
Estimate the sales that are at least necessary to reach 0 profit as the difference between low sales and the low_profit. We set the 'low_profit' to zero and for the average and high profit we substract the sales that are at least necessary to reach 0 from the average sales and the high sales.

In [120]:
ep_data.loc[ep_data['hh_n40']<0,'low_profit']=0
ep_data.loc[ep_data['hh_n40']<0,'average_profit']=ep_data.loc[ep_data['hh_n40']<0, 'hh_n34']-(ep_data.loc[ep_data['hh_n40']<0, 'hh_n32']-ep_data.loc[ep_data['hh_n40']<0, 'hh_n40'])
ep_data.loc[ep_data['hh_n40']<0,'high_profit']= ep_data.loc[ep_data['hh_n40']<0,'average_profit']*ep_data.loc[ep_data['hh_n40']<0, 'high_sales_fac']

In [121]:
ep_data.loc[ep_data['average_profit']<0]

,case_id,hh_n13a,hh_n13b,hh_n25a,hh_n25b,hh_n25c,hh_n25d,hh_n25e,hh_n25f,hh_n25g,...,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac,low_profit,high_profit,average_profit
3907,204212290095,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-7200.0,0.833333,1.416667,0.0,-8783.333333,-6200.0


In [122]:
ep_data['average_profit'].iloc[3907]=0
ep_data['high_profit'].iloc[3907]=0
ep_data.iloc[3907,:]

/home/insauer/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


case_id           2.042123e+11
hh_n13a           0.000000e+00
hh_n13b           0.000000e+00
hh_n25a                    NaN
hh_n25b                    NaN
hh_n25c                    NaN
hh_n25d                    NaN
hh_n25e                    NaN
hh_n25f                    NaN
hh_n25g                    NaN
hh_n25h           2.000000e+00
hh_n25i           2.000000e+00
hh_n25j           3.000000e+00
hh_n25k           3.000000e+00
hh_n25l           1.000000e+00
hh_n25m           0.000000e+00
hh_n25n                    NaN
hh_n25o                    NaN
hh_n25p                    NaN
hh_n25q                    NaN
hh_n25r                    NaN
hh_n25s                    NaN
hh_n25t                    NaN
hh_n25u                    NaN
hh_n25v                    NaN
hh_n25w                    NaN
hh_n25x                    NaN
hh_n25y                    NaN
hh_n32            5.000000e+03
hh_n33            1.000000e+00
hh_n34            6.000000e+03
hh_n35            8.500000e+03
hh_n36  

In [123]:
ep_data.iloc[:,20:]

,hh_n25r,hh_n25s,hh_n25t,hh_n25u,hh_n25v,hh_n25w,hh_n25x,hh_n25y,hh_n32,hh_n33,...,hh_n36,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac,low_profit,high_profit,average_profit
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1200.0,1.0,...,NaN,NaN,NaN,NaN,500.0,0.400000,1.566667,500.000000,1958.333333,1250.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,700.0,1.0,...,NaN,NaN,NaN,NaN,-350.0,0.466667,1.666667,0.000000,750.000000,450.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
op_data = ep_data[['hh_n25a', 'hh_n25b', 'hh_n25c', 'hh_n25d', 'hh_n25e', 'hh_n25f', 'hh_n25g',
                   'hh_n25h', 'hh_n25i', 'hh_n25j','hh_n25k', 'hh_n25l', 'hh_n25m', 'hh_n25n', 'hh_n25o', 'hh_n25p',
                   'hh_n25q', 'hh_n25r', 'hh_n25s', 'hh_n25t', 'hh_n25u', 'hh_n25v', 'hh_n25w', 'hh_n25x', 'hh_n25y']]

In [125]:
op_data #= op_data.transpose()

,hh_n25a,hh_n25b,hh_n25c,hh_n25d,hh_n25e,hh_n25f,hh_n25g,hh_n25h,hh_n25i,hh_n25j,...,hh_n25p,hh_n25q,hh_n25r,hh_n25s,hh_n25t,hh_n25u,hh_n25v,hh_n25w,hh_n25x,hh_n25y
0,NaN,3.0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
op_data = op_data.transpose()

In [127]:
op_data

,0,1,2,3,4,5,6,7,8,9,...,12499,12500,12501,12502,12503,12504,12505,12506,12507,12508
hh_n25a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hh_n25b,3.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hh_n25c,3.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hh_n25d,3.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hh_n25e,2.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hh_n25f,2.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,...,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
hh_n25g,2.0,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,...,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,0.0
hh_n25h,2.0,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,...,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,0.0
hh_n25i,1.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN,NaN,NaN,...,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,0.0
hh_n25j,1.0,NaN,NaN,NaN,NaN,1.0,3.0,NaN,NaN,NaN,...,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,0.0


In [128]:
ep_data['last_month'] = op_data.apply(pd.Series.last_valid_index)

In [129]:
ep_data

,case_id,hh_n13a,hh_n13b,hh_n25a,hh_n25b,hh_n25c,hh_n25d,hh_n25e,hh_n25f,hh_n25g,...,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac,low_profit,high_profit,average_profit,last_month
0,101010160009,0.0,0.0,NaN,3.0,3.0,3.0,2.0,2.0,2.0,...,NaN,NaN,NaN,500.0,0.400000,1.566667,500.000000,1958.333333,1250.0,hh_n25n
1,101010160033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,101010160060,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-350.0,0.466667,1.666667,0.000000,750.000000,450.0,hh_n25m
3,101010160068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,101010160069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,315556140342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
12505,315556140365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
12506,315556140388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
12507,315556140411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [130]:
def select_col(x):
    
    month_cols = ['hh_n25a', 'hh_n25b', 'hh_n25c', 'hh_n25d', 'hh_n25e', 'hh_n25f', 'hh_n25g',
                   'hh_n25h', 'hh_n25i', 'hh_n25j','hh_n25k', 'hh_n25l', 'hh_n25m', 'hh_n25n', 'hh_n25o', 'hh_n25p',
                   'hh_n25q', 'hh_n25r', 'hh_n25s', 'hh_n25t', 'hh_n25u', 'hh_n25v', 'hh_n25w', 'hh_n25x', 'hh_n25y']
    
    co_owners = x['hh_n13a'] + x['hh_n13b'] + 1
    
    if not x['last_month'] is None:
        last_prod = x[x['last_month']]
    else:
        last_prod = np.nan

    n_month = 0
    productivity=['low_profit','average_profit', 'high_profit']
    profit = 0
    for month in month_cols:
        if not (np.isnan(x[month]) or x[month]==0. or x[month]>3.0):
            profit+=x[productivity[int(x[month]-1)]]
            n_month+=1
    
    if n_month >12:
        annual_profit = (profit/n_month)*12
    else:
        annual_profit = profit
    weekly_profit = annual_profit/52
    
    annual_profit_pcoo = annual_profit/co_owners
    
    weekly_profit_pcoo = weekly_profit/co_owners
    
    return pd.Series([x['case_id'],
                      last_prod,
                      n_month,
                      annual_profit,
                      weekly_profit,
                      annual_profit_pcoo,
                      weekly_profit_pcoo],
                     index=['case_id',
                            'last_prod',
                            'n_month',
                            'annual_profit',
                            'weekly_profit',
                            'annual_profit_pcoo',
                            'weekly_profit_pcoo'])

In [131]:
ep_fin_data = ep_data.apply(select_col, axis=1)

In [132]:
ep_data.iloc[:,5:]

,hh_n25c,hh_n25d,hh_n25e,hh_n25f,hh_n25g,hh_n25h,hh_n25i,hh_n25j,hh_n25k,hh_n25l,...,hh_n37,hh_n38,hh_n39,hh_n40,low_sales_fac,high_sales_fac,low_profit,high_profit,average_profit,last_month
0,3.0,3.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,500.0,0.400000,1.566667,500.000000,1958.333333,1250.0,hh_n25n
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,...,NaN,NaN,NaN,-350.0,0.466667,1.666667,0.000000,750.000000,450.0,hh_n25m
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
12505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
12506,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
12507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [133]:
ep_fin_data

,case_id,last_prod,n_month,annual_profit,weekly_profit,annual_profit_pcoo,weekly_profit_pcoo
0,1.010102e+11,1.0,13.0,12807.692308,246.301775,12807.692308,246.301775
1,1.010102e+11,NaN,0.0,0.000000,0.000000,NaN,NaN
2,1.010102e+11,1.0,3.0,1200.000000,23.076923,1200.000000,23.076923
3,1.010102e+11,NaN,0.0,0.000000,0.000000,NaN,NaN
4,1.010102e+11,NaN,0.0,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...
12504,3.155561e+11,NaN,0.0,0.000000,0.000000,NaN,NaN
12505,3.155561e+11,NaN,0.0,0.000000,0.000000,NaN,NaN
12506,3.155561e+11,NaN,0.0,0.000000,0.000000,NaN,NaN
12507,3.155561e+11,NaN,0.0,0.000000,0.000000,NaN,NaN


In [134]:
ep_act = ep_fin_data.groupby(['case_id']).sum()

In [135]:
ep_act['case_id']=ep_act.index

In [136]:
ep_act

,last_prod,n_month,annual_profit,weekly_profit,annual_profit_pcoo,weekly_profit_pcoo,case_id
case_id,,,,,,,
1.010102e+11,1.0,13.0,12807.692308,246.301775,12807.692308,246.301775,1.010102e+11
1.010102e+11,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.010102e+11
1.010102e+11,1.0,3.0,1200.000000,23.076923,1200.000000,23.076923,1.010102e+11
1.010102e+11,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.010102e+11
1.010102e+11,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.010102e+11
...,...,...,...,...,...,...,...
3.155561e+11,0.0,0.0,0.000000,0.000000,0.000000,0.000000,3.155561e+11
3.155561e+11,0.0,0.0,0.000000,0.000000,0.000000,0.000000,3.155561e+11
3.155561e+11,0.0,0.0,0.000000,0.000000,0.000000,0.000000,3.155561e+11


In [137]:
ep_act['case_id']=ep_act['case_id'].astype(int)


In [138]:
ep_act['case_id2']= np.array(general_info['case_id'])

In [139]:
ep_act['check']=ep_act['case_id']-ep_act['case_id2']

In [140]:
ep_act.loc[ep_act['check']!=0]

,last_prod,n_month,annual_profit,weekly_profit,annual_profit_pcoo,weekly_profit_pcoo,case_id,case_id2,check
case_id,,,,,,,,,


In [141]:
general_info['income_enterprise']=np.array(ep_act['annual_profit_pcoo'])

In [142]:
general_info

,case_id,ea_id,isurban,weight,region,income_labour,income_enterprise,inc-sp_child,inc-sp_other,income_other
0,101010160009,10101016,2,126.560097,101,2400.0,12807.692308,0.0,1500.0,0.0
1,101010160033,10101016,2,126.560097,101,0.0,0.000000,2000.0,18000.0,0.0
2,101010160060,10101016,2,126.560097,101,84480.0,1200.000000,20000.0,6000.0,0.0
3,101010160068,10101016,2,126.560097,101,16080.0,0.000000,0.0,0.0,0.0
4,101010160069,10101016,2,126.560097,101,2100.0,0.000000,0.0,250.0,0.0
...,...,...,...,...,...,...,...,...,...,...
12266,315556140342,31555614,1,456.450592,315,90000.0,0.000000,0.0,1500.0,0.0
12267,315556140365,31555614,1,456.450592,315,85680.0,0.000000,0.0,0.0,0.0
12268,315556140388,31555614,1,456.450592,315,72000.0,0.000000,0.0,0.0,0.0
12269,315556140411,31555614,1,456.450592,315,91000.0,0.000000,0.0,0.0,0.0


# Social spendings
Income from children elsewhere. They can live either in Malawi or abroad. We consider the money as income from social transfers and consider it to come from inside the country.

In [143]:
ch_data = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Household/hh_mod_o.csv')

In [144]:
ch_data = ch_data[['case_id','hh_o17']]

In [145]:
ep_act = ch_data.groupby(['case_id']).sum()

In [146]:
ep_act

,hh_o17
case_id,
101010160009,0.0
101010160033,2000.0
101010160060,20000.0
101010160068,0.0
101010160069,0.0
...,...
315556140342,0.0
315556140365,0.0
315556140388,0.0


In [147]:
general_info['inc-sp_child']=np.array(ep_act['hh_o17'])

In [148]:
general_info

,case_id,ea_id,isurban,weight,region,income_labour,income_enterprise,inc-sp_child,inc-sp_other,income_other
0,101010160009,10101016,2,126.560097,101,2400.0,12807.692308,0.0,1500.0,0.0
1,101010160033,10101016,2,126.560097,101,0.0,0.000000,2000.0,18000.0,0.0
2,101010160060,10101016,2,126.560097,101,84480.0,1200.000000,20000.0,6000.0,0.0
3,101010160068,10101016,2,126.560097,101,16080.0,0.000000,0.0,0.0,0.0
4,101010160069,10101016,2,126.560097,101,2100.0,0.000000,0.0,250.0,0.0
...,...,...,...,...,...,...,...,...,...,...
12266,315556140342,31555614,1,456.450592,315,90000.0,0.000000,0.0,1500.0,0.0
12267,315556140365,31555614,1,456.450592,315,85680.0,0.000000,0.0,0.0,0.0
12268,315556140388,31555614,1,456.450592,315,72000.0,0.000000,0.0,0.0,0.0
12269,315556140411,31555614,1,456.450592,315,91000.0,0.000000,0.0,0.0,0.0


# Other income

In [149]:
oi_data = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Household/hh_mod_p.csv')

In [150]:
oi_data

,case_id,visit,ea_id,hh_p0a,hh_p0a_os,hh_p01,hh_p02,hh_p03a,hh_p03b,hh_p03c,hh_p04a,hh_p04b
0,101010160009,1,10101016,101.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,101010160009,1,10101016,102.0,NaN,1.0,NaN,1500.0,0.0,0.0,1.0,NaN
2,101010160009,1,10101016,103.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,101010160009,1,10101016,104.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,101010160009,1,10101016,105.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
184061,315556140434,2,31555614,111.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
184062,315556140434,2,31555614,112.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
184063,315556140434,2,31555614,113.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
184064,315556140434,2,31555614,114.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
codes_soc = [101, 102, 103, 105]
soc_oi_data = oi_data.loc[oi_data['hh_p0a'].isin(codes_soc)]

In [152]:
soc_oi_data = soc_oi_data[['case_id', 'hh_p02', 'hh_p03a', 'hh_p03b', 'hh_p03c']]

In [153]:
soc_oi_data_fin = soc_oi_data.groupby(['case_id']).sum()
soc_oi_data_fin['inc-sp_other']=soc_oi_data_fin.sum(axis=1)
soc_oi_data_fin

,hh_p02,hh_p03a,hh_p03b,hh_p03c,inc-sp_other
case_id,,,,,
101010160009,0.0,1500.0,0.0,0.0,1500.0
101010160033,0.0,0.0,18000.0,0.0,18000.0
101010160060,0.0,0.0,6000.0,0.0,6000.0
101010160068,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,250.0,0.0,0.0,250.0
...,...,...,...,...,...
315556140342,0.0,0.0,1500.0,0.0,1500.0
315556140365,0.0,0.0,0.0,0.0,0.0
315556140388,0.0,0.0,0.0,0.0,0.0


In [154]:
general_info['inc-sp_other']=np.array(soc_oi_data_fin['inc-sp_other'])
general_info

,case_id,ea_id,isurban,weight,region,income_labour,income_enterprise,inc-sp_child,inc-sp_other,income_other
0,101010160009,10101016,2,126.560097,101,2400.0,12807.692308,0.0,1500.0,0.0
1,101010160033,10101016,2,126.560097,101,0.0,0.000000,2000.0,18000.0,0.0
2,101010160060,10101016,2,126.560097,101,84480.0,1200.000000,20000.0,6000.0,0.0
3,101010160068,10101016,2,126.560097,101,16080.0,0.000000,0.0,0.0,0.0
4,101010160069,10101016,2,126.560097,101,2100.0,0.000000,0.0,250.0,0.0
...,...,...,...,...,...,...,...,...,...,...
12266,315556140342,31555614,1,456.450592,315,90000.0,0.000000,0.0,1500.0,0.0
12267,315556140365,31555614,1,456.450592,315,85680.0,0.000000,0.0,0.0,0.0
12268,315556140388,31555614,1,456.450592,315,72000.0,0.000000,0.0,0.0,0.0
12269,315556140411,31555614,1,456.450592,315,91000.0,0.000000,0.0,0.0,0.0


In [155]:
codes_inc = [104, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115]
inc_oi_data = oi_data.loc[oi_data['hh_p0a'].isin(codes_inc)]
inc_oi_data = inc_oi_data[['case_id', 'hh_p02', 'hh_p03a', 'hh_p03b', 'hh_p03c']]

In [156]:
inc_oi_data

,case_id,hh_p02,hh_p03a,hh_p03b,hh_p03c
3,101010160009,NaN,NaN,NaN,NaN
5,101010160009,NaN,NaN,NaN,NaN
6,101010160009,NaN,NaN,NaN,NaN
7,101010160009,NaN,NaN,NaN,NaN
8,101010160009,NaN,NaN,NaN,NaN
...,...,...,...,...,...
184061,315556140434,NaN,NaN,NaN,NaN
184062,315556140434,NaN,NaN,NaN,NaN
184063,315556140434,NaN,NaN,NaN,NaN
184064,315556140434,NaN,NaN,NaN,NaN


In [157]:
inc_oi_data_fin = inc_oi_data.groupby(['case_id']).sum()
inc_oi_data_fin

,hh_p02,hh_p03a,hh_p03b,hh_p03c
case_id,,,,
101010160009,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0
...,...,...,...,...
315556140342,0.0,0.0,0.0,0.0
315556140365,0.0,0.0,0.0,0.0
315556140388,0.0,0.0,0.0,0.0


In [158]:
inc_oi_data_fin['income_other']=inc_oi_data_fin.sum(axis=1)
inc_oi_data_fin

,hh_p02,hh_p03a,hh_p03b,hh_p03c,income_other
case_id,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
315556140342,0.0,0.0,0.0,0.0,0.0
315556140365,0.0,0.0,0.0,0.0,0.0
315556140388,0.0,0.0,0.0,0.0,0.0


In [159]:
general_info['income_other']=np.array(inc_oi_data_fin['income_other'])
general_info.head(50)

,case_id,ea_id,isurban,weight,region,income_labour,income_enterprise,inc-sp_child,inc-sp_other,income_other
0,101010160009,10101016,2,126.560097,101,2400.0,12807.692308,0.0,1500.0,0.0
1,101010160033,10101016,2,126.560097,101,0.0,0.000000,2000.0,18000.0,0.0
2,101010160060,10101016,2,126.560097,101,84480.0,1200.000000,20000.0,6000.0,0.0
3,101010160068,10101016,2,126.560097,101,16080.0,0.000000,0.0,0.0,0.0
4,101010160069,10101016,2,126.560097,101,2100.0,0.000000,0.0,250.0,0.0
5,101010160070,10101016,2,126.560097,101,129000.0,70200.000000,1500.0,0.0,0.0
6,101010160074,10101016,2,126.560097,101,134400.0,11951.428571,0.0,0.0,5000.0
7,101010160086,10101016,2,126.560097,101,277032.0,0.000000,0.0,0.0,0.0
8,101010160096,10101016,2,126.560097,101,1920.0,0.000000,0.0,0.0,0.0
9,101010160143,10101016,2,126.560097,101,4000.0,0.000000,0.0,0.0,0.0


# Income social safety nets

In [160]:
sn_data = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Household/hh_mod_r.csv')

In [161]:
sn_data

,case_id,visit,ea_id,hh_r0a,hh_r0a_os,hh_r01,hh_r02a,hh_r02b,hh_r02c,hh_r03,hh_r04a,hh_r04b,hh_r04c,hh_r04d,hh_r04e,hh_r05a,hh_r05b,hh_r06,hh_r07a,hh_r07b
0,101010160009,1,10101016,101,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101010160009,1,10101016,102,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101010160009,1,10101016,103,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101010160009,1,10101016,104,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101010160009,1,10101016,105,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159518,315556140434,2,31555614,109,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159519,315556140434,2,31555614,110,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159520,315556140434,2,31555614,111,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159521,315556140434,2,31555614,112,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
codes = [102, 103, 104, 108, 109, 110, 111, 112]
sn_data = sn_data.loc[sn_data['hh_r0a'].isin(codes)]
sn_data

,case_id,visit,ea_id,hh_r0a,hh_r0a_os,hh_r01,hh_r02a,hh_r02b,hh_r02c,hh_r03,hh_r04a,hh_r04b,hh_r04c,hh_r04d,hh_r04e,hh_r05a,hh_r05b,hh_r06,hh_r07a,hh_r07b
1,101010160009,1,10101016,102,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101010160009,1,10101016,103,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101010160009,1,10101016,104,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,101010160009,1,10101016,108,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,101010160009,1,10101016,109,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159517,315556140434,2,31555614,108,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159518,315556140434,2,31555614,109,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159519,315556140434,2,31555614,110,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159520,315556140434,2,31555614,111,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
sn_data=sn_data[['case_id', 'hh_r02a', 'hh_r02b']]

In [164]:
sn_data['sum_ss'] = sn_data[['hh_r02a', 'hh_r02b']].sum(axis=1)

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [165]:
sn_data.head(50)

,case_id,hh_r02a,hh_r02b,sum_ss
1,101010160009,NaN,NaN,0.0
2,101010160009,NaN,NaN,0.0
3,101010160009,NaN,NaN,0.0
7,101010160009,NaN,NaN,0.0
8,101010160009,NaN,NaN,0.0
9,101010160009,NaN,NaN,0.0
10,101010160009,NaN,NaN,0.0
11,101010160009,NaN,NaN,0.0
14,101010160033,NaN,NaN,0.0
15,101010160033,NaN,NaN,0.0


In [166]:
sn_data_fin = sn_data.groupby(['case_id']).sum()
sn_data_fin#.head(50)

,hh_r02a,hh_r02b,sum_ss
case_id,,,
101010160009,0.0,0.0,0.0
101010160033,0.0,0.0,0.0
101010160060,0.0,0.0,0.0
101010160068,0.0,0.0,0.0
101010160069,0.0,0.0,0.0
...,...,...,...
315556140342,0.0,0.0,0.0
315556140365,0.0,0.0,0.0
315556140388,0.0,0.0,0.0


In [167]:
general_info['inc-sp_ssn']=np.array(sn_data_fin['sum_ss'])
general_info.head(50)

,case_id,ea_id,isurban,weight,region,income_labour,income_enterprise,inc-sp_child,inc-sp_other,income_other,inc-sp_ssn
0,101010160009,10101016,2,126.560097,101,2400.0,12807.692308,0.0,1500.0,0.0,0.0
1,101010160033,10101016,2,126.560097,101,0.0,0.000000,2000.0,18000.0,0.0,0.0
2,101010160060,10101016,2,126.560097,101,84480.0,1200.000000,20000.0,6000.0,0.0,0.0
3,101010160068,10101016,2,126.560097,101,16080.0,0.000000,0.0,0.0,0.0,0.0
4,101010160069,10101016,2,126.560097,101,2100.0,0.000000,0.0,250.0,0.0,0.0
5,101010160070,10101016,2,126.560097,101,129000.0,70200.000000,1500.0,0.0,0.0,13500.0
6,101010160074,10101016,2,126.560097,101,134400.0,11951.428571,0.0,0.0,5000.0,0.0
7,101010160086,10101016,2,126.560097,101,277032.0,0.000000,0.0,0.0,0.0,0.0
8,101010160096,10101016,2,126.560097,101,1920.0,0.000000,0.0,0.0,0.0,0.0
9,101010160143,10101016,2,126.560097,101,4000.0,0.000000,0.0,0.0,0.0,0.0


In [168]:
general_info.index=general_info['case_id']

In [169]:
general_info

,case_id,ea_id,isurban,weight,region,income_labour,income_enterprise,inc-sp_child,inc-sp_other,income_other,inc-sp_ssn
case_id,,,,,,,,,,,
101010160009,101010160009,10101016,2,126.560097,101,2400.0,12807.692308,0.0,1500.0,0.0,0.0
101010160033,101010160033,10101016,2,126.560097,101,0.0,0.000000,2000.0,18000.0,0.0,0.0
101010160060,101010160060,10101016,2,126.560097,101,84480.0,1200.000000,20000.0,6000.0,0.0,0.0
101010160068,101010160068,10101016,2,126.560097,101,16080.0,0.000000,0.0,0.0,0.0,0.0
101010160069,101010160069,10101016,2,126.560097,101,2100.0,0.000000,0.0,250.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
315556140342,315556140342,31555614,1,456.450592,315,90000.0,0.000000,0.0,1500.0,0.0,0.0
315556140365,315556140365,31555614,1,456.450592,315,85680.0,0.000000,0.0,0.0,0.0,0.0
315556140388,315556140388,31555614,1,456.450592,315,72000.0,0.000000,0.0,0.0,0.0,0.0


In [170]:
general_info.to_csv('/home/insauer/projects/WB_model/Data_Malawi/general_info.csv')

# Income from agricultural activities

### Rainy season

Rainy season:

income:

ag_b08 --> value of all cropsales (In module b there are only around 5000 hhs so we use module i --> difference not clear)

ag_d17a --> income from plot rent

ag_d17b --> income from plot rent

ag_d19a --> income from plot rent

ag_d19b --> income from plot rent

ag_d19c --> income from plot rent

ag_d19d --> income from plot rent

ag_i03 --> value of all cropsales

expenses:

ag_b10 --> sales transport costs (In module b there are only around 5000 hhs so we use module i --> difference not clear)

ag_d09a --> payment for plot (in kind)

ag_d09b --> payment for plot (cash)

ag_d11a --> payment for plot

ag_d11b --> payment for plot

ag_d11c --> payment for plot

ag_d11d --> payment for plot

ag_d46a	--> hired men days

ag_d46b	--> hired men wage

ag_d46c	--> hired women days

ag_d46d	--> hired women wage

ag_d46e	--> hired child days

ag_d46f --> hired child wage

ag_f09 --> transport input

ag_f10 --> value input

ag_h09 --> transport seeds

ag_h10 --> value seeds

ag_h40 --> value seeds transport

ag_i10 --> sales transport costs

In [172]:
agb_data = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_b.csv')
agb_data

,case_id,visit,ea_id,ag_b0a,ag_b0c,ag_b0c_os,ag_b01a,ag_b01b,ag_b02,ag_b03a,...,ag_b27b_os,ag_b27c,ag_b28a,ag_b28b,ag_b28b_os,ag_b28c,ag_b29a,ag_b29b,ag_b29b_os,ag_b29c
0,101012150001,1,10101215,1.0,1.0,NaN,1.00,1.0,2.0,NaN,...,NaN,NaN,1.0,2.0,NaN,1.0,0.0,NaN,NaN,NaN
1,101012150001,1,10101215,1.0,11.0,NaN,0.25,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101012150001,1,10101215,1.0,38.0,NaN,0.25,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101012150004,1,10101215,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101012150021,1,10101215,1.0,1.0,NaN,1.00,1.0,2.0,NaN,...,NaN,NaN,8.0,5.0,NaN,1.0,4.0,5.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5404,315546130283,1,31554613,1.0,34.0,NaN,0.50,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5405,315556140113,1,31555614,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5406,315556140136,1,31555614,1.0,1.0,NaN,0.60,1.0,2.0,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5407,315556140159,1,31555614,1.0,3.0,NaN,1.00,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check if all hh in the ag survey are also in the hh-survey

In [173]:
ag_cases = list(set(agb_data['case_id']))

In [174]:
len(ag_cases)

2666

In [175]:
sn_cases = list(sn_data_fin.index)

In [176]:
len(sn_cases)

12271

In [177]:
for i in ag_cases:
    test = i in sn_cases
    print(test)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


...seems to be the case

We know collect the input costs for farming first and then the output and earnings

input modules:



In [178]:
module_b = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_b.csv')
module_b = module_b[['case_id', 'ag_b04a', 'ag_b04b', 'ag_b07a', 'ag_b07b', 'ag_b07c', 'ag_b08', 'ag_b10',
                     'ag_b16a', 'ag_b16b', 'ag_b16c', 'ag_b16d']]
#module_b.iloc[:,15:25]
#module_b['ag_b08'].dropna()
#module_b[['case_id','ag_b08']]#.dropna()
module_b

,case_id,ag_b04a,ag_b04b,ag_b07a,ag_b07b,ag_b07c,ag_b08,ag_b10,ag_b16a,ag_b16b,ag_b16c,ag_b16d
0,101012150001,8.0,2.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
1,101012150001,1.0,2.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,101012150001,6.0,5.0,4.0,5.0,1.0,1800.0,0.0,0.0,NaN,NaN,NaN
3,101012150004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101012150021,4.0,3.0,3.0,5.0,1.0,1800.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5404,315546130283,25.0,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
5405,315556140113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5406,315556140136,7.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1.0,14.0
5407,315556140159,16.0,2.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [179]:
agr_summary = module_b[['case_id', 'ag_b08', 'ag_b10']]

In [180]:
agr_summary = agr_summary.rename(columns={'ag_b08': 'income_sell_rainy', 'ag_b10': 'exp_transp_rainy' })

In [181]:
agr_summary[~np.isnan(agr_summary['income_sell_rainy'])]

,case_id,income_sell_rainy,exp_transp_rainy
2,101012150001,1800.0,0.0
4,101012150021,1800.0,0.0
5,101012150021,400.0,0.0
7,101012150022,2100.0,0.0
8,101012150022,9400.0,0.0
...,...,...,...
5359,314431080099,20000.0,0.0
5364,314431080159,3000.0,0.0
5366,314431080165,2000.0,0.0
5370,314432700023,2400.0,0.0


In [182]:
agr_summary.index=agr_summary['case_id']

In [183]:
agr_summary= agr_summary.iloc[:,1:]

In [184]:
agr_summary.head(50)

,income_sell_rainy,exp_transp_rainy
case_id,,
101012150001,NaN,NaN
101012150001,NaN,NaN
101012150001,1800.0,0.0
101012150004,NaN,NaN
101012150021,1800.0,0.0
101012150021,400.0,0.0
101012150022,NaN,NaN
101012150022,2100.0,0.0
101012150022,9400.0,0.0


In [185]:
agr_summary = agr_summary.groupby(['case_id']).sum()

In [186]:
agr_summary

,income_sell_rainy,exp_transp_rainy
case_id,,
101010160009,28500.0,1800.0
101010160033,0.0,0.0
101010160060,16000.0,0.0
101010160068,27400.0,1300.0
101010160069,7350.0,600.0
...,...,...
315546130283,0.0,0.0
315556140113,0.0,0.0
315556140136,0.0,0.0


In [187]:
module_d = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_d.csv')
module_d = module_d[['case_id','ag_d09a', 'ag_d09b', 'ag_d11a', 'ag_d11b', 'ag_d11c', 'ag_d11d', 'ag_d17a',
          'ag_d17b', 'ag_d19a', 'ag_d19b', 'ag_d19c', 'ag_d19d', 'ag_d46a', 'ag_d46b', 'ag_d46c','ag_d46d', 'ag_d46e', 'ag_d46f'
         ]]

module_d['hired_paym'] = module_d['ag_d46a'].replace(np.nan,0)*module_d['ag_d46b'].replace(np.nan,0) +\
                           module_d['ag_d46c'].replace(np.nan,0)*module_d['ag_d46d'].replace(np.nan,0) +\
                           module_d['ag_d46e'].replace(np.nan,0)*module_d['ag_d46f'].replace(np.nan,0)
#module_b.iloc[:,15:25]
module_d['hired_paym']

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (23,30,34,62,64,68,72,76,78,85,90,93,96,103,108,113,211) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
19260    0.0
19261    0.0
19262    0.0
19263    0.0
19264    0.0
Name: hired_paym, Length: 19265, dtype: float64

In [188]:
mod_d = module_d.groupby(['case_id']).sum()
mod_d

,ag_d09a,ag_d09b,ag_d11a,ag_d11b,ag_d11c,ag_d11d,ag_d17a,ag_d17b,ag_d19a,ag_d19b,ag_d19c,ag_d19d,ag_d46a,ag_d46b,ag_d46c,ag_d46d,ag_d46e,ag_d46f,hired_paym
case_id,,,,,,,,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [189]:
inc_df = mod_d[[ 'ag_d17a', 'ag_d17b', 'ag_d19a', 'ag_d19b', 'ag_d19c', 'ag_d19d']]
inc_df

,ag_d17a,ag_d17b,ag_d19a,ag_d19b,ag_d19c,ag_d19d
case_id,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0


In [190]:
inc_df['income_pltrent'] = inc_df.sum(axis=1)

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [191]:
inc_df

,ag_d17a,ag_d17b,ag_d19a,ag_d19b,ag_d19c,ag_d19d,income_pltrent
case_id,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
exp_df= mod_d[[ 'ag_d09a', 'ag_d09b', 'ag_d11a', 'ag_d11b', 'ag_d11c', 'ag_d11d', 'hired_paym']]

In [193]:
exp_df['exp_pltrent_hiredwrk'] = exp_df.sum(axis=1)

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [194]:
exp_df

,ag_d09a,ag_d09b,ag_d11a,ag_d11b,ag_d11c,ag_d11d,hired_paym,exp_pltrent_hiredwrk
case_id,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [195]:
agr_summary

,income_sell_rainy,exp_transp_rainy
case_id,,
101010160009,28500.0,1800.0
101010160033,0.0,0.0
101010160060,16000.0,0.0
101010160068,27400.0,1300.0
101010160069,7350.0,600.0
...,...,...
315546130283,0.0,0.0
315556140113,0.0,0.0
315556140136,0.0,0.0


In [196]:
agr_info = inc_df[['income_pltrent']]
agr_info['exp_pltrent_hiredwrk']=exp_df['exp_pltrent_hiredwrk']
agr_info

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,income_pltrent,exp_pltrent_hiredwrk
case_id,,
101010160009,0.0,0.0
101010160033,0.0,0.0
101010160060,0.0,0.0
101010160068,0.0,0.0
101010160069,0.0,0.0
...,...,...
315546130283,0.0,0.0
315556140113,0.0,0.0
315556140136,0.0,0.0


In [197]:
agr_info['income_sell_rainy']=agr_summary['income_sell_rainy']

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [198]:
agr_info['income_sell_rainy']=agr_summary['income_sell_rainy']
agr_info['exp_transp_rainy']=agr_summary['exp_transp_rainy']

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [199]:
agr_info=agr_info.rename(columns={'income_pltrent':'income_pltrent_rainy',
                                  'exp_pltrent_hiredwrk': 'exp_pltrent_hiredwrk_rainy'})

In [200]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,income_sell_rainy,exp_transp_rainy
case_id,,,,
101010160009,0.0,0.0,28500.0,1800.0
101010160033,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,16000.0,0.0
101010160068,0.0,0.0,27400.0,1300.0
101010160069,0.0,0.0,7350.0,600.0
...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0


In [201]:
# input 
module_f = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_f.csv')
module_f = module_f[['case_id', 'ag_f09', 'ag_f10' ]]

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5,8,14,17,25,30,51,53,60,62,71) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [202]:
mod_f = module_f.groupby(['case_id']).sum()
mod_f

,ag_f09,ag_f10
case_id,,
101010160009,0.0,0.0
101010160033,0.0,0.0
101010160060,0.0,0.0
101010160068,0.0,0.0
101010160069,0.0,0.0
...,...,...
315546130283,0.0,0.0
315556140113,0.0,0.0
315556140136,0.0,0.0


In [203]:
agr_info['exp_transp_inp_rainy']=mod_f['ag_f09']
agr_info['exp_inp_rainy']=mod_f['ag_f10']

In [205]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,income_sell_rainy,exp_transp_rainy,exp_transp_inp_rainy,exp_inp_rainy
case_id,,,,,,
101010160009,0.0,0.0,28500.0,1800.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,16000.0,0.0,0.0,0.0
101010160068,0.0,0.0,27400.0,1300.0,0.0,0.0
101010160069,0.0,0.0,7350.0,600.0,0.0,0.0
...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0


In [297]:
# input 
module_g = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_g.csv')
module_g = module_g[['case_id','ag_g0a', 'ag_g0b', 'ag_g0d', 'ag_g08a', 'ag_g08b', 'ag_g08b_os', 'ag_g09a',
                     'ag_g09b', 'ag_g09b_os']]

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We convert units roughly to liters

KILOGRAM........1

50 KG BAG.......2

90 KG BAG.......3

PAIL (SMALL)....4

PAIL (LARGE)....5

NO. 10 PLATE....6

NO. 12 PLATE....7

BUNCH...........8

PIECE...........9

BALE...........10

BASKET (DENGU).11

OX-CART........12

In [298]:
def convert_unit(x):
    unit_conv= {1.0: 1,
               2.0:50,
               3.0:90,
               4.0:8,
               5.0:20,
               6.0:1,
               7.0:1.5,
               8.0:1,
               9.0:1,
               10.0:100,
               11.0:40,
               12.0:1000,
               0.0: 0.0,
               13.0: 0.0
               }
    return pd.Series(unit_conv[x],
                     index=['unit_8'])

In [299]:
module_g

,case_id,ag_g0a,ag_g0b,ag_g0d,ag_g08a,ag_g08b,ag_g08b_os,ag_g09a,ag_g09b,ag_g09b_os
0,101012150001,1.0,R1,1.0,4.0,2.0,NaN,0.0,NaN,NaN
1,101012150001,1.0,R2,1.0,3.0,5.0,NaN,4.0,5.0,NaN
2,101012150001,1.0,R2,11.0,2.0,5.0,NaN,1.0,5.0,NaN
3,101012150001,1.0,R3,3.0,1.0,2.0,NaN,0.0,NaN,NaN
4,101012150001,1.0,R2,38.0,2.0,2.0,NaN,1.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...
28138,315556140113,NaN,R1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
28139,315556140113,NaN,R1,11.0,NaN,NaN,NaN,NaN,NaN,NaN
28140,315556140136,NaN,R1,1.0,NaN,NaN,NaN,NaN,NaN,NaN
28141,315556140159,NaN,R1,3.0,NaN,NaN,NaN,NaN,NaN,NaN


In [300]:
module_g['unit_8']=module_g['ag_g08b'].replace(np.nan,0.0).apply((convert_unit))

In [301]:
module_g.loc[(module_g['ag_g09b']==13.0)]

,case_id,ag_g0a,ag_g0b,ag_g0d,ag_g08a,ag_g08b,ag_g08b_os,ag_g09a,ag_g09b,ag_g09b_os,unit_8
728,101041500012,1.0,R1,1.0,NaN,NaN,NaN,1.0,13.0,ONE GRANARY NKHOKWE,0.0
737,101041500048,1.0,R1,1.0,NaN,NaN,NaN,2.0,13.0,FOR GRANARIES,0.0
751,101041500067,1.0,R1,1.0,NaN,NaN,NaN,1.0,13.0,ONE GRANARY NKHOKWE,0.0
754,101041500079,1.0,R1,1.0,NaN,NaN,NaN,1.0,13.0,ONE GRANARY NKHOKWE,0.0
7394,204073980199,1.0,R1,4.0,NaN,NaN,NaN,4.0,13.0,3 BASKETS,0.0
8246,206016260092,1.0,R4,27.0,NaN,NaN,NaN,9.0,13.0,TON,0.0
8250,206016260104,1.0,R1,3.0,NaN,NaN,NaN,2.0,13.0,TON,0.0
8251,206016260116,1.0,R1,3.0,NaN,NaN,NaN,2.0,13.0,TON,0.0
8252,206016260116,1.0,R3,35.0,NaN,NaN,NaN,5.0,13.0,TON,0.0
8253,206016260116,1.0,R2,11.0,NaN,NaN,NaN,2.0,13.0,TON,0.0


In [302]:
module_g.loc[(module_g['ag_g08b']==13.0) & (module_g['ag_g08b_os']=='TON'), 'unit_8']=1000
module_g.loc[(module_g['ag_g08b']==13.0) & (module_g['ag_g08b_os']=='BASIN'), 'unit_8']=30
module_g.loc[(module_g['ag_g08b']==13.0) & (module_g['ag_g08b_os']=='EXPECT TO HARVEST ONE NDOWA'), 'unit_8']=100


In [303]:
module_g['unit_9']=module_g['ag_g09b'].replace(np.nan,0.0).apply((convert_unit))

In [304]:
module_g

,case_id,ag_g0a,ag_g0b,ag_g0d,ag_g08a,ag_g08b,ag_g08b_os,ag_g09a,ag_g09b,ag_g09b_os,unit_8,unit_9
0,101012150001,1.0,R1,1.0,4.0,2.0,NaN,0.0,NaN,NaN,50.0,0.0
1,101012150001,1.0,R2,1.0,3.0,5.0,NaN,4.0,5.0,NaN,20.0,20.0
2,101012150001,1.0,R2,11.0,2.0,5.0,NaN,1.0,5.0,NaN,20.0,20.0
3,101012150001,1.0,R3,3.0,1.0,2.0,NaN,0.0,NaN,NaN,50.0,0.0
4,101012150001,1.0,R2,38.0,2.0,2.0,NaN,1.0,2.0,NaN,50.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...
28138,315556140113,NaN,R1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
28139,315556140113,NaN,R1,11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
28140,315556140136,NaN,R1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
28141,315556140159,NaN,R1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [305]:
module_g.loc[(module_g['ag_g09b']==13.0)&(module_g['ag_g09b_os']=='TON'),'unit_9']=1000
module_g.loc[(module_g['ag_g09b']==13.0)&(module_g['ag_g09b_os']=='ONE GRANARY NKHOKWE'),'unit_9']=3000
module_g.loc[(module_g['ag_g09b']==13.0)&(module_g['ag_g09b_os']=='FOR GRANARY'),'unit_9']=12000
module_g.loc[(module_g['ag_g09b']==13.0)&(module_g['ag_g09b_os']=='3 BASKETS'),'unit_9']=90
module_g.loc[(module_g['ag_g09b']==13.0)&(module_g['ag_g09b_os']=='NOT YET HARVESTED'),'unit_9']=40
module_g.loc[(module_g['ag_g09b']==13.0)&(module_g['ag_g09b_os']=='PIECES'),'unit_9']=20
module_g.loc[(module_g['ag_g09b']==13.0)&(module_g['ag_g09b_os']=='DENGU'),'unit_9']=40
module_g.loc[(module_g['ag_g09b']==13.0)&(module_g['ag_g09b_os']=='LICHELO'),'unit_9']=5

In [306]:
module_g.loc[(module_g['ag_g09b']==13.0)]

,case_id,ag_g0a,ag_g0b,ag_g0d,ag_g08a,ag_g08b,ag_g08b_os,ag_g09a,ag_g09b,ag_g09b_os,unit_8,unit_9
728,101041500012,1.0,R1,1.0,NaN,NaN,NaN,1.0,13.0,ONE GRANARY NKHOKWE,0.0,3000.0
737,101041500048,1.0,R1,1.0,NaN,NaN,NaN,2.0,13.0,FOR GRANARIES,0.0,0.0
751,101041500067,1.0,R1,1.0,NaN,NaN,NaN,1.0,13.0,ONE GRANARY NKHOKWE,0.0,3000.0
754,101041500079,1.0,R1,1.0,NaN,NaN,NaN,1.0,13.0,ONE GRANARY NKHOKWE,0.0,3000.0
7394,204073980199,1.0,R1,4.0,NaN,NaN,NaN,4.0,13.0,3 BASKETS,0.0,90.0
8246,206016260092,1.0,R4,27.0,NaN,NaN,NaN,9.0,13.0,TON,0.0,1000.0
8250,206016260104,1.0,R1,3.0,NaN,NaN,NaN,2.0,13.0,TON,0.0,1000.0
8251,206016260116,1.0,R1,3.0,NaN,NaN,NaN,2.0,13.0,TON,0.0,1000.0
8252,206016260116,1.0,R3,35.0,NaN,NaN,NaN,5.0,13.0,TON,0.0,1000.0
8253,206016260116,1.0,R2,11.0,NaN,NaN,NaN,2.0,13.0,TON,0.0,1000.0


In [307]:
module_g['harvested']=module_g['ag_g08a'].replace(np.nan,0) * module_g['unit_8'] + module_g['ag_g09a'].replace(np.nan,0) * module_g['unit_9']

In [308]:
module_g

,case_id,ag_g0a,ag_g0b,ag_g0d,ag_g08a,ag_g08b,ag_g08b_os,ag_g09a,ag_g09b,ag_g09b_os,unit_8,unit_9,harvested
0,101012150001,1.0,R1,1.0,4.0,2.0,NaN,0.0,NaN,NaN,50.0,0.0,200.0
1,101012150001,1.0,R2,1.0,3.0,5.0,NaN,4.0,5.0,NaN,20.0,20.0,140.0
2,101012150001,1.0,R2,11.0,2.0,5.0,NaN,1.0,5.0,NaN,20.0,20.0,60.0
3,101012150001,1.0,R3,3.0,1.0,2.0,NaN,0.0,NaN,NaN,50.0,0.0,50.0
4,101012150001,1.0,R2,38.0,2.0,2.0,NaN,1.0,2.0,NaN,50.0,50.0,150.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28138,315556140113,NaN,R1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
28139,315556140113,NaN,R1,11.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
28140,315556140136,NaN,R1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
28141,315556140159,NaN,R1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [328]:
def aggregate_harvest(x):
    harv_sum= x['harvested'].sum()
    crop_type=x['ag_g0d'].iloc[0]
    return pd.Series([crop_type,harv_sum],
                     index=['crop','harvest'])

In [329]:
harvest_rainy=module_g.groupby(['case_id','ag_g0d'])[['ag_g0d','harvested']].apply(aggregate_harvest).reset_index()

In [336]:
harvest_rainy=harvest_rainy.drop(columns=['ag_g0d'])

In [360]:
harvest_rainy#.loc[harvest_rainy['case_id']==101012150001]

,case_id,crop,harvest
0,101010160009,2.0,300.0
1,101010160009,5.0,200.0
2,101010160009,12.0,100.0
3,101010160033,1.0,270.0
4,101010160060,3.0,180.0
...,...,...,...
23548,315556140113,1.0,0.0
23549,315556140113,11.0,0.0
23550,315556140136,1.0,0.0
23551,315556140159,3.0,0.0


In [87]:
#check = mod_d.index == mod_i.index
#check[check==False]

array([], dtype=bool)

In [206]:
module_h = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_h.csv')
module_h = module_h[['case_id', 'ag_h09', 'ag_h10', 'ag_h40']]

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (16,28,38,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [207]:
mod_h = module_h.groupby(['case_id']).sum()
mod_h

,ag_h09,ag_h10,ag_h40
case_id,,,
101010160009,0.0,0.0,0.0
101010160033,0.0,0.0,0.0
101010160060,0.0,0.0,0.0
101010160068,0.0,0.0,0.0
101010160069,0.0,0.0,0.0
...,...,...,...
315546130283,0.0,0.0,0.0
315556140113,0.0,0.0,0.0
315556140136,0.0,0.0,0.0


In [208]:
agr_info['exp_transp_seed_rainy']=mod_h['ag_h09'] + mod_h['ag_h40'] 
agr_info['exp_seed_rainy']=mod_h['ag_h10']

In [209]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,income_sell_rainy,exp_transp_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy
case_id,,,,,,,,
101010160009,0.0,0.0,28500.0,1800.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,16000.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,27400.0,1300.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,7350.0,600.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [340]:
module_i = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_i.csv')
#module_i[['case_id', 'ag_i03', 'ag_i10']]
module_i = module_i[['case_id', 'ag_i0b', 'ag_i0b_os', 'ag_i01', 'ag_i02a', 'ag_i02b', 'ag_i02b_os', 'ag_i02c', 'ag_i03', 'ag_i10']]#.dropna()

In [361]:
module_i

,case_id,ag_i0b,ag_i0b_os,ag_i01,ag_i02a,ag_i02b,ag_i02b_os,ag_i02c,ag_i03,ag_i10
0,101012150001,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,101012150001,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,101012150001,11.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,101012150001,38.0,NaN,1.0,5.0,5.0,NaN,1.0,1500.0,NaN
4,101012150004,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
24076,315556140113,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
24077,315556140113,11.0,NaN,1.0,10.0,1.0,NaN,2.0,700.0,NaN
24078,315556140136,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
24079,315556140159,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [363]:
mod_i_cut=module_i.loc[module_i['case_id'].isin(harvest_rainy['case_id'])]

In [371]:
mod_i_cut.index = mod_i_cut['case_id']
mod_i_cut

,case_id,ag_i0b,ag_i0b_os,ag_i01,ag_i02a,ag_i02b,ag_i02b_os,ag_i02c,ag_i03,ag_i10
case_id,,,,,,,,,,
101012150001,101012150001,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
101012150001,101012150001,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
101012150001,101012150001,11.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
101012150001,101012150001,38.0,NaN,1.0,5.0,5.0,NaN,1.0,1500.0,NaN
101012150004,101012150004,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
315556140113,315556140113,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
315556140113,315556140113,11.0,NaN,1.0,10.0,1.0,NaN,2.0,700.0,NaN
315556140136,315556140136,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [341]:
#mod_i = module_i.groupby(['case_id']).sum()
#mod_i

In [680]:
agr_info['income_sell_rainy_fin'] = mod_i['ag_i03']
agr_info['exp_transp_rainy_fin'] = mod_i['ag_i10']

In [681]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,income_sell_rainy,exp_transp_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sell_rainy_fin,exp_transp_rainy_fin
case_id,,,,,,,,,,
101010160009,0.0,0.0,28500.0,1800.0,0.0,0.0,0.0,0.0,40000.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0
101010160060,0.0,0.0,16000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,27400.0,1300.0,0.0,0.0,0.0,0.0,62850.0,0.0
101010160069,0.0,0.0,7350.0,600.0,0.0,0.0,0.0,0.0,3300.0,0.0
...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [682]:
agr_info.loc[agr_info['income_sell_rainy']<agr_info['income_sell_rainy_fin'],'income_sales_rainy']= agr_info.loc[agr_info['income_sell_rainy']<agr_info['income_sell_rainy_fin'],'income_sell_rainy_fin']
agr_info.loc[agr_info['income_sell_rainy']>=agr_info['income_sell_rainy_fin'],'income_sales_rainy']= agr_info.loc[agr_info['income_sell_rainy']>=agr_info['income_sell_rainy_fin'],'income_sell_rainy']

agr_info.loc[agr_info['exp_transp_rainy']<agr_info['exp_transp_rainy_fin'],'exp_trnsp_rainy']= agr_info.loc[agr_info['exp_transp_rainy']<agr_info['exp_transp_rainy_fin'],'exp_transp_rainy_fin']
agr_info.loc[agr_info['exp_transp_rainy']>=agr_info['exp_transp_rainy_fin'],'exp_trnsp_rainy']= agr_info.loc[agr_info['exp_transp_rainy']>=agr_info['exp_transp_rainy_fin'],'exp_transp_rainy']

In [683]:
agr_info.drop(columns=['income_sell_rainy','income_sell_rainy_fin','exp_transp_rainy','exp_transp_rainy_fin'])

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy
case_id,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0
...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [684]:
agr_info['income_rainy'] = agr_info[['income_pltrent_rainy', 'income_sales_rainy']].sum(axis=1)
agr_info['exp_rainy'] = agr_info[['exp_pltrent_hiredwrk_rainy', 'exp_transp_inp_rainy',
                                  'exp_inp_rainy', 'exp_transp_seed_rainy', 'exp_seed_rainy']].sum(axis=1)

In [685]:
agr_info = agr_info.drop(columns=['income_sell_rainy','income_sell_rainy_fin','exp_transp_rainy','exp_transp_rainy_fin'])

In [686]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy
case_id,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0,40000.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0,16000.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0,62850.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0,7350.0,0.0
...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,700.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Dimba season

Dimba season

income:

ag_k18a --> payment plot

ag_k18b --> payment plot

ag_k20a --> payment plot

ag_k20b --> payment plot

ag_k20c --> payment plot

ag_k20d --> payment plot

expenditure:

ag_k10a --> payment plot

ag_k10b --> payment plot

ag_k12a --> payment plot

ag_k12b --> payment plot

ag_k12c --> payment plot

ag_k12d --> payment plot

ag_k46a --> days hired men

ag_k46b --> payment hired men

ag_k46c --> days hired women

ag_k46d --> payment hired women

ag_k46e --> days hired children

ag_k46f --> payment hired women

In [687]:
module_k = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_k.csv')

module_k = module_k[['case_id','ag_k10a', 'ag_k10b', 'ag_k12a', 'ag_k12b', 'ag_k12c', 'ag_k12d',
                     'ag_k18a', 'ag_k18b', 'ag_k20a', 'ag_k20b', 'ag_k20c', 'ag_k20d',
                     'ag_k46a', 'ag_k46b', 'ag_k46c', 'ag_k46d', 'ag_k46e', 'ag_k46f']]

module_k

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,33,35,54,56,60,68,72,80,89,96,99,106,114,119) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_id,ag_k10a,ag_k10b,ag_k12a,ag_k12b,ag_k12c,ag_k12d,ag_k18a,ag_k18b,ag_k20a,ag_k20b,ag_k20c,ag_k20d,ag_k46a,ag_k46b,ag_k46c,ag_k46d,ag_k46e,ag_k46f
0,101012150001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,101012150004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101012150021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101012150022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101012150026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13581,315546130283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13582,315556140113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13583,315556140136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13584,315556140159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [688]:
module_k['hired_paym'] = module_k['ag_k46a'].replace(np.nan,0)*module_k['ag_k46b'].replace(np.nan,0) +\
                           module_k['ag_k46c'].replace(np.nan,0)*module_k['ag_k46d'].replace(np.nan,0) +\
                           module_k['ag_k46e'].replace(np.nan,0)*module_k['ag_k46f'].replace(np.nan,0)

In [689]:
module_k['exp_pltrent_hiredwrk_dimba']= module_k['ag_k10a'].replace(np.nan,0)+module_k['ag_k10b'].replace(np.nan,0)+\
                                        module_k['ag_k12a'].replace(np.nan,0)+module_k['ag_k12b'].replace(np.nan,0)+\
                                        module_k['ag_k12c'].replace(np.nan,0)+module_k['ag_k12d'].replace(np.nan,0)+\
                                        module_k['hired_paym'].replace(np.nan,0)
module_k['income_pltrent_dimba']= module_k['ag_k18a'].replace(np.nan,0)+module_k['ag_k18b'].replace(np.nan,0)+\
                                        module_k['ag_k20a'].replace(np.nan,0)+module_k['ag_k20b'].replace(np.nan,0)+\
                                        module_k['ag_k20c'].replace(np.nan,0)+module_k['ag_k20d'].replace(np.nan,0)

In [690]:
module_k

,case_id,ag_k10a,ag_k10b,ag_k12a,ag_k12b,ag_k12c,ag_k12d,ag_k18a,ag_k18b,ag_k20a,...,ag_k20d,ag_k46a,ag_k46b,ag_k46c,ag_k46d,ag_k46e,ag_k46f,hired_paym,exp_pltrent_hiredwrk_dimba,income_pltrent_dimba
0,101012150001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,101012150004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,101012150021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
3,101012150022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,101012150026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13581,315546130283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
13582,315556140113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
13583,315556140136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
13584,315556140159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [691]:
mod_k = module_k.groupby(['case_id']).sum()
mod_k

,ag_k10a,ag_k10b,ag_k12a,ag_k12b,ag_k12c,ag_k12d,ag_k18a,ag_k18b,ag_k20a,ag_k20b,...,ag_k20d,ag_k46a,ag_k46b,ag_k46c,ag_k46d,ag_k46e,ag_k46f,hired_paym,exp_pltrent_hiredwrk_dimba,income_pltrent_dimba
case_id,,,,,,,,,,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [692]:
agr_info['exp_pltrent_hiredwrk_dimba']=mod_k['exp_pltrent_hiredwrk_dimba']
agr_info['income_pltrent_dimba']=mod_k['income_pltrent_dimba']

In [693]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy,exp_pltrent_hiredwrk_dimba,income_pltrent_dimba
case_id,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0,40000.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0,16000.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0,62850.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0,7350.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,700.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [694]:
module_l = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_l.csv')
module_l = module_l[['case_id','ag_l09', 'ag_l10', 'ag_l40']]
module_l

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (25,32,38,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_id,ag_l09,ag_l10,ag_l40
0,101012150001,NaN,NaN,NaN
1,101012150004,NaN,NaN,NaN
2,101012150021,NaN,NaN,NaN
3,101012150022,NaN,NaN,NaN
4,101012150026,NaN,NaN,NaN
...,...,...,...,...
11378,315546130283,NaN,NaN,NaN
11379,315556140113,NaN,NaN,NaN
11380,315556140136,NaN,NaN,NaN
11381,315556140159,NaN,NaN,NaN


In [695]:
mod_l = module_l.groupby(['case_id']).sum()
mod_l

,ag_l09,ag_l10,ag_l40
case_id,,,
101010160009,0.0,0.0,0.0
101010160033,0.0,0.0,0.0
101010160060,0.0,0.0,0.0
101010160068,0.0,0.0,0.0
101010160069,0.0,0.0,0.0
...,...,...,...
315546130283,0.0,0.0,0.0
315556140113,0.0,0.0,0.0
315556140136,0.0,0.0,0.0


In [696]:
agr_info['exp_transp_inp_dimba']=mod_l['ag_l09']+mod_l['ag_l40']
agr_info['exp_inp_dimba']=mod_l['ag_l10']

In [697]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy,exp_pltrent_hiredwrk_dimba,income_pltrent_dimba,exp_transp_inp_dimba,exp_inp_dimba
case_id,,,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0,40000.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0,16000.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0,62850.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0,7350.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,700.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [698]:
module_n = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_n.csv')
module_n = module_n[['case_id','ag_n09', 'ag_n10', 'ag_n40']]
module_n

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (13,24,37) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_id,ag_n09,ag_n10,ag_n40
0,101012150001,NaN,NaN,NaN
1,101012150004,NaN,NaN,NaN
2,101012150021,NaN,NaN,NaN
3,101012150022,NaN,NaN,NaN
4,101012150026,NaN,NaN,NaN
...,...,...,...,...
10950,315546130283,NaN,NaN,NaN
10951,315556140113,NaN,NaN,NaN
10952,315556140136,NaN,NaN,NaN
10953,315556140159,NaN,NaN,NaN


In [699]:
mod_n = module_n.groupby(['case_id']).sum()
mod_n

,ag_n09,ag_n10,ag_n40
case_id,,,
101010160009,0.0,0.0,0.0
101010160033,0.0,0.0,0.0
101010160060,0.0,0.0,0.0
101010160068,0.0,0.0,0.0
101010160069,0.0,0.0,0.0
...,...,...,...
315546130283,0.0,0.0,0.0
315556140113,0.0,0.0,0.0
315556140136,0.0,0.0,0.0


In [700]:
agr_info['exp_transp_seed_dimba']=mod_n['ag_n09']+mod_n['ag_n40']
agr_info['exp_seed_dimba']=mod_n['ag_n10']

In [701]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy,exp_pltrent_hiredwrk_dimba,income_pltrent_dimba,exp_transp_inp_dimba,exp_inp_dimba,exp_transp_seed_dimba,exp_seed_dimba
case_id,,,,,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0,16000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0,62850.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0,7350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [702]:
module_o = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_o.csv')
module_o = module_o[['case_id','ag_o03', 'ag_o10']]
module_o

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,37,85,91,96,98,100) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_id,ag_o03,ag_o10
0,101012150001,NaN,NaN
1,101012150004,NaN,NaN
2,101012150021,NaN,NaN
3,101012150022,NaN,NaN
4,101012150026,NaN,NaN
...,...,...,...
10948,315546130283,NaN,NaN
10949,315556140113,NaN,NaN
10950,315556140136,NaN,NaN
10951,315556140159,NaN,NaN


In [703]:
mod_o = module_o.groupby(['case_id']).sum()
mod_o

,ag_o03,ag_o10
case_id,,
101010160009,0.0,0.0
101010160033,0.0,0.0
101010160060,0.0,0.0
101010160068,0.0,0.0
101010160069,0.0,0.0
...,...,...
315546130283,0.0,0.0
315556140113,0.0,0.0
315556140136,0.0,0.0


In [704]:
agr_info['income_sales_dimba']=mod_o['ag_o03']
agr_info['exp_trnsp_dimba']=mod_o['ag_o10']

In [705]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy,exp_pltrent_hiredwrk_dimba,income_pltrent_dimba,exp_transp_inp_dimba,exp_inp_dimba,exp_transp_seed_dimba,exp_seed_dimba,income_sales_dimba,exp_trnsp_dimba
case_id,,,,,,,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0,16000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0,62850.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0,7350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,700.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [706]:
agr_info['income_dimba']=agr_info['income_pltrent_dimba']+agr_info['income_sales_dimba']
agr_info['exp_dimba']= agr_info['exp_pltrent_hiredwrk_dimba']+agr_info['exp_transp_inp_dimba']+agr_info['exp_inp_dimba']+\
                        agr_info['exp_transp_seed_dimba']+agr_info['exp_seed_dimba']

In [708]:
agr_info.loc[agr_info['income_dimba']>0]

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy,exp_pltrent_hiredwrk_dimba,income_pltrent_dimba,exp_transp_inp_dimba,exp_inp_dimba,exp_transp_seed_dimba,exp_seed_dimba,income_sales_dimba,exp_trnsp_dimba,income_dimba,exp_dimba
case_id,,,,,,,,,,,,,,,,,,,,
101012040062,0.0,0.0,0.0,0.0,0.0,0.0,39000.0,0.0,39000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,1000.0,0.0
101012150074,0.0,0.0,0.0,0.0,0.0,0.0,25000.0,0.0,25000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1500.0,0.0,1500.0,0.0
101026210018,0.0,0.0,600.0,12500.0,0.0,1550.0,NaN,NaN,0.0,14650.0,0.0,0.0,0.0,0.0,0.0,0.0,17700.0,0.0,17700.0,0.0
101026210024,0.0,0.0,0.0,750.0,0.0,1750.0,NaN,NaN,0.0,2500.0,0.0,0.0,0.0,1100.0,0.0,0.0,22000.0,0.0,22000.0,1100.0
101026210042,0.0,0.0,1600.0,54800.0,0.0,9250.0,NaN,NaN,0.0,65650.0,2100.0,0.0,0.0,6400.0,0.0,0.0,13500.0,0.0,13500.0,8500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313065000071,0.0,6300.0,0.0,0.0,750.0,750.0,NaN,NaN,0.0,7800.0,0.0,0.0,0.0,750.0,0.0,180.0,8600.0,0.0,8600.0,930.0
313065000078,0.0,1400.0,1200.0,14200.0,0.0,4000.0,NaN,NaN,0.0,20800.0,0.0,0.0,0.0,200.0,0.0,0.0,2000.0,0.0,2000.0,200.0
313067190008,0.0,0.0,2000.0,18000.0,0.0,2300.0,NaN,NaN,0.0,22300.0,0.0,0.0,0.0,650.0,0.0,120.0,9000.0,0.0,9000.0,770.0


### Trees and permanent crops

income:

ag_q03 --> income from sales 


expenditure:

ag_q03 --> payment sales transport


In [709]:
module_q = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_q.csv')
module_q = module_q[['case_id','ag_q03', 'ag_q10']]
module_q

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (50,61,72,89,93) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_id,ag_q03,ag_q10
0,101012150001,NaN,NaN
1,101012150004,NaN,NaN
2,101012150021,NaN,NaN
3,101012150022,NaN,NaN
4,101012150026,NaN,NaN
...,...,...,...
12325,315546130283,NaN,NaN
12326,315556140113,NaN,NaN
12327,315556140136,800.0,NaN
12328,315556140159,NaN,NaN


In [710]:
mod_q = module_q.groupby(['case_id']).sum()
mod_q

,ag_q03,ag_q10
case_id,,
101010160009,0.0,0.0
101010160033,0.0,0.0
101010160060,0.0,0.0
101010160068,10000.0,0.0
101010160069,0.0,0.0
...,...,...
315546130283,0.0,0.0
315556140113,0.0,0.0
315556140136,800.0,0.0


In [711]:
agr_info['income_sales_tree']=mod_q['ag_q03']
agr_info['exp_trnsp_tree']=mod_q['ag_q10']
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy,...,exp_transp_inp_dimba,exp_inp_dimba,exp_transp_seed_dimba,exp_seed_dimba,income_sales_dimba,exp_trnsp_dimba,income_dimba,exp_dimba,income_sales_tree,exp_trnsp_tree
case_id,,,,,,,,,,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0,40000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0,16000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0,62850.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0,7350.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,700.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0


In [712]:
agr_info['income_sales_tree']=mod_q['ag_q03']
agr_info['exp_trnsp_tree']=mod_q['ag_q10']
agr_info['income_tree']=agr_info['income_sales_tree']
agr_info['exp_tree']=agr_info['exp_trnsp_tree']

In [713]:
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy,...,exp_transp_seed_dimba,exp_seed_dimba,income_sales_dimba,exp_trnsp_dimba,income_dimba,exp_dimba,income_sales_tree,exp_trnsp_tree,income_tree,exp_tree
case_id,,,,,,,,,,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0,40000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0,16000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0,62850.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10000.0,0.0,10000.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0,7350.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,700.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,800.0,0.0,800.0,0.0


### Livestock

income:

ag_r17 --> income from sales

ag_s06 --> product sales


expenditure:

ag_r11 --> purchases

ag_r27 --> hired labor

ag_r28 --> animal feed

ag_r29 --> medicine

ag_r30 --> vet service

ag_r31 --> other input


ag_s13 --> additional expenses


In [722]:
module_r1 = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_r1.csv')
module_r1 = module_r1[['case_id','ag_r11', 'ag_r17']]
module_r1

/home/insauer/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,case_id,ag_r11,ag_r17
0,101012150001,NaN,NaN
1,101012150004,NaN,NaN
2,101012150004,NaN,NaN
3,101012150004,NaN,NaN
4,101012150004,NaN,NaN
...,...,...,...
107534,315556140113,NaN,NaN
107535,315556140113,NaN,NaN
107536,315556140136,NaN,NaN
107537,315556140159,NaN,NaN


In [723]:
mod_r1 = module_r1.groupby(['case_id']).sum()
mod_r1

,ag_r11,ag_r17
case_id,,
101010160009,1000.0,0.0
101010160033,0.0,0.0
101010160060,16000.0,4500.0
101010160068,1700.0,800.0
101010160069,0.0,0.0
...,...,...
315546130283,0.0,0.0
315556140113,1000.0,0.0
315556140136,0.0,0.0


In [716]:
module_r2 = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_r2.csv')
module_r2 = module_r2[['case_id','ag_r27', 'ag_r28', 'ag_r29', 'ag_r30', 'ag_r31']]
module_r2

,case_id,ag_r27,ag_r28,ag_r29,ag_r30,ag_r31
0,101012150001,NaN,NaN,NaN,NaN,NaN
1,101012150004,0.0,0.0,100.0,0.0,0.0
2,101012150021,0.0,0.0,0.0,0.0,0.0
3,101012150022,0.0,0.0,0.0,0.0,0.0
4,101012150026,800.0,0.0,600.0,0.0,0.0
...,...,...,...,...,...,...
10396,315546130283,NaN,NaN,NaN,NaN,NaN
10397,315556140113,0.0,0.0,0.0,0.0,0.0
10398,315556140136,NaN,NaN,NaN,NaN,NaN
10399,315556140159,NaN,NaN,NaN,NaN,NaN


In [724]:
mod_r2 = module_r2.groupby(['case_id']).sum()
mod_r2['ls_care'] = mod_r2.sum(axis=1)

In [725]:
mod_r2

,ag_r27,ag_r28,ag_r29,ag_r30,ag_r31,ls_care
case_id,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,3000.0,300.0,0.0,0.0,3300.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0


In [726]:
agr_info['income_livestock_sales']=mod_r1['ag_r11']
agr_info['exp_livestock_purchase']=mod_r1['ag_r17']
agr_info['exp_livestock_care']=mod_r2['ls_care']
agr_info

,income_pltrent_rainy,exp_pltrent_hiredwrk_rainy,exp_transp_inp_rainy,exp_inp_rainy,exp_transp_seed_rainy,exp_seed_rainy,income_sales_rainy,exp_trnsp_rainy,income_rainy,exp_rainy,...,exp_trnsp_dimba,income_dimba,exp_dimba,income_sales_tree,exp_trnsp_tree,income_tree,exp_tree,inc_livestock_sales,exp_livestock_purchase,exp_livestock_care
case_id,,,,,,,,,,,,,,,,,,,,,
101010160009,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,1800.0,40000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0
101010160033,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101010160060,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,0.0,16000.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,4500.0,3300.0
101010160068,0.0,0.0,0.0,0.0,0.0,0.0,62850.0,1300.0,62850.0,0.0,...,0.0,0.0,0.0,10000.0,0.0,10000.0,0.0,1700.0,800.0,0.0
101010160069,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,600.0,7350.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,0.0,0.0,0.0,0.0,0.0,0.0,700.0,0.0,700.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,0.0
315556140136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,800.0,0.0,800.0,0.0,0.0,0.0,0.0


In [720]:
module_s = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Agriculture/ag_mod_s.csv')
module_s = module_s[['case_id', 'ag_s06', 'ag_s13']]
module_s

,case_id,ag_s06,ag_s13
0,101012150001,NaN,NaN
1,101012150004,NaN,NaN
2,101012150004,NaN,NaN
3,101012150004,NaN,NaN
4,101012150004,NaN,NaN
...,...,...,...
50394,315556140113,NaN,NaN
50395,315556140113,NaN,NaN
50396,315556140136,NaN,NaN
50397,315556140159,NaN,NaN


In [727]:
mod_s = module_s.groupby(['case_id']).sum()
mod_s

,ag_s06,ag_s13
case_id,,
101010160009,800.0,0.0
101010160033,0.0,0.0
101010160060,4000.0,0.0
101010160068,0.0,0.0
101010160069,0.0,0.0
...,...,...
315546130283,0.0,0.0
315556140113,0.0,0.0
315556140136,0.0,0.0


In [738]:
agr_info['income_livestock_prod_sales']=mod_s['ag_s06']
agr_info['exp_livestock_add_exp']=mod_s['ag_s13']

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [754]:
agr_info['income_livestock']=agr_info['income_livestock_sales']+agr_info['income_livestock_prod_sales']
agr_info['exp_livestock']=agr_info['exp_livestock_purchase']+agr_info['exp_livestock_care']+agr_info['exp_livestock_add_exp']

In [755]:
agr_short=agr_info[['income_rainy', 'exp_rainy', 'income_dimba','exp_dimba', 'income_tree', 'exp_tree', 'income_livestock', 'exp_livestock']]

In [756]:
agr_short['total_income']=agr_info['income_rainy']+agr_info['income_dimba']+agr_info['income_tree']+ agr_info['income_livestock']
agr_short['total_exp']=agr_info['exp_rainy']+agr_info['exp_dimba']+agr_info['exp_tree']+ agr_info['exp_livestock']
agr_short['total_profit']=agr_short['total_income']-agr_short['total_exp']

/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [763]:
agr_short.loc[agr_short['total_profit']<0.0,'total_profit_equalized']= 0.0
agr_short.loc[agr_short['total_profit']>=0.0,'total_profit_equalized']=agr_short['total_profit']

/home/insauer/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/insauer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [764]:
agr_short

,income_rainy,exp_rainy,income_dimba,exp_dimba,income_tree,exp_tree,income_livestock,exp_livestock,total_income,total_exp,total_profit,total_profit_equalized
case_id,,,,,,,,,,,,
101010160009,40000.0,0.0,0.0,0.0,0.0,0.0,1800.0,0.0,41800.0,0.0,41800.0,41800.0
101010160033,900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,900.0,0.0,900.0,900.0
101010160060,16000.0,0.0,0.0,0.0,0.0,0.0,20000.0,7800.0,36000.0,7800.0,28200.0,28200.0
101010160068,62850.0,0.0,0.0,0.0,10000.0,0.0,1700.0,800.0,74550.0,800.0,73750.0,73750.0
101010160069,7350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7350.0,0.0,7350.0,7350.0
...,...,...,...,...,...,...,...,...,...,...,...,...
315546130283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315556140113,700.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.0,1700.0,0.0,1700.0,1700.0
315556140136,0.0,0.0,0.0,0.0,800.0,0.0,0.0,0.0,800.0,0.0,800.0,800.0


In [784]:
agr_short.loc[agr_short['exp_tree']>agr_short['income_tree']]

,income_rainy,exp_rainy,income_dimba,exp_dimba,income_tree,exp_tree,income_livestock,exp_livestock,total_income,total_exp,total_profit,total_profit_equalized
case_id,,,,,,,,,,,,


In [765]:
general_info['income_agr']=agr_short['total_profit_equalized']

In [768]:
general_info['income_agr']=general_info['income_agr'].replace(np.nan,0.0)

In [769]:
general_info

,case_id,ea_id,isurban,weight,region,income_labour,income_enterprise,inc-sp_child,inc-sp_other,income_other,inc-sp_ssn,income_agr
case_id,,,,,,,,,,,,
101010160009,101010160009,10101016,2,126.560097,101,2400.0,12807.692308,0.0,1500.0,0.0,0.0,41800.0
101010160033,101010160033,10101016,2,126.560097,101,0.0,0.000000,2000.0,18000.0,0.0,0.0,900.0
101010160060,101010160060,10101016,2,126.560097,101,84480.0,1200.000000,20000.0,6000.0,0.0,0.0,28200.0
101010160068,101010160068,10101016,2,126.560097,101,16080.0,0.000000,0.0,0.0,0.0,0.0,73750.0
101010160069,101010160069,10101016,2,126.560097,101,2100.0,0.000000,0.0,250.0,0.0,0.0,7350.0
...,...,...,...,...,...,...,...,...,...,...,...,...
315556140342,315556140342,31555614,1,456.450592,315,90000.0,0.000000,0.0,1500.0,0.0,0.0,0.0
315556140365,315556140365,31555614,1,456.450592,315,85680.0,0.000000,0.0,0.0,0.0,0.0,0.0
315556140388,315556140388,31555614,1,456.450592,315,72000.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [771]:
general_info['income']=general_info.iloc[:,5:].sum(axis=1)

In [774]:
general_info.loc[general_info['income']==0]

,case_id,ea_id,isurban,weight,region,income_labour,income_enterprise,inc-sp_child,inc-sp_other,income_other,inc-sp_ssn,income_agr,income
case_id,,,,,,,,,,,,,
101012190018,101012190018,10101219,2,134.102707,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101012190025,101012190025,10101219,2,134.102707,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101012190092,101012190092,10101219,2,134.102707,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101013550014,101013550014,10101355,2,75.261101,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101013550032,101013550032,10101355,2,75.261101,101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
315311610291,315311610291,31531161,1,541.124329,315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315326740121,315326740121,31532674,1,647.444885,315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
315404190147,315404190147,31540419,1,512.634094,315,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Income from fisheries activities

In [777]:
modulef_d1 = pd.read_csv('/home/insauer/projects/WB_model/Data_Malawi/MWI_2010_IHS-III_v01_M_CSV/Full_Sample/Fisheries/fs_mod_d1.csv')
modulef_d1 = module_sd1[['case_id', 'ag_s06', 'ag_s13']]
modulef_d1


,case_id,visit,ea_id,fs_d0a,fs_d0a_os,fs_d01,fs_d02,fs_d03,fs_d04,fs_d05,fs_d06
0,101034140064,2,10103414,1,NaN,2.0,NaN,NaN,NaN,NaN,NaN
1,101034140064,2,10103414,2,NaN,2.0,NaN,NaN,NaN,NaN,NaN
2,101034140064,2,10103414,3,NaN,2.0,NaN,NaN,NaN,NaN,NaN
3,101034140064,2,10103414,4,NaN,1.0,1.0,0.0,20000.0,0.0,0.0
4,101034140064,2,10103414,5,NaN,2.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1867,314426160009,1,31442616,4,NaN,2.0,NaN,NaN,NaN,NaN,NaN
1868,314426160009,1,31442616,5,NaN,2.0,NaN,NaN,NaN,NaN,NaN
1869,314426160009,1,31442616,6,NaN,2.0,NaN,NaN,NaN,NaN,NaN
1870,314426160009,1,31442616,7,NaN,2.0,NaN,NaN,NaN,NaN,NaN
